## 算例参数

In [ ]:
# imports
import numpy as np
from pyDOE import lhs
from numpy import genfromtxt
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn as sk
from sklearn import preprocessing
from sklearn.preprocessing import MinMaxScaler
import math
import timeit
from ipywidgets import interactive
import tensorflow as tf
import tensorflow.keras.backend as kb
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Concatenate, Flatten, Dense, Conv2D, MaxPool2D, Dropout, BatchNormalization, Lambda, ReLU
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.activations import relu, softmax, tanh
from tensorflow.keras import regularizers
from tensorflow.keras.losses import mean_squared_error
from tensorflow.keras.initializers import he_uniform, glorot_normal, zeros, ones
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

tf.keras.backend.set_floatx('float64')

In [ ]:
T0 = 0
T20 = 20

# 水的密度rhow [kg/m^3]
T_crit = 374.15                           # [℃]
b0 = 1.0213E3
b1 = -7.7377E-1
b2 = 8.7696E-3
b3 = -9.2118E-5
b4 = 3.3534E-7
b5 = -4.4034E-10
pw1 = 1e7                                 # [Pa]
pwrif = 2e7                                # [Pa]
a0 = 4.8863E-7
a1 = -1.6528E-9
a2 = 1.8621E-12
a3 = 2.4266E-13
a4 = -1.5996E-15
a5 = 3.3703E-18

# 水汽密度rhogw [kg/m^3]
c1 = -5.8002206E3
c2 = 1.3914993
c3 = -4.8640239E-2
c4 = 4.1764768E-5
c5 = -1.4452093E-8
c6 = 6.5459673
molar_w = 18.0152E-3                      # M_w [kg/mol]
rgas = 8.31/1e6                                # R [J/(mol K)]
Kelvin = 273.15                           # 开尔文系数[K]

# 干空气密度rhoga [kg/m^3]
molar_a = 28.95E-3                        # M_a [kg/mol]
A_pg = 1                                 

# 混合气体动态粘度etag [Pa s]
T0 = 0
etagw0 = 8.85E-6/1e6                           # [Pa s]
etaga0 = 17.17E-6/1e6                          # [Pa s]
agw = 3.633E-8/1e6                             # [Pa s/℃]
aga = 4.733E-8/1e6                             # [Pa s/℃]
bga = -2.222E-11/1e6                          # [Pa s/℃^2]
bg = 0.608

# 水相动态黏度etaw [Pa s]
aw = 6.612E-1/1e6  
bw = -1.562
T_etaw = 44.15

# 混凝土孔隙率poro [-] 
poro0 = 0.1  
A_n = 1e-4                                # [1/℃]

# 本征渗透系数k  [m^2] 
akint = 5.157E-16                             # [m^2]
bkint = 5.322E-3                          # [℃^(-1)]
bsf = 1e5/1e6                                   # [Pa]

# 饱和度Sw  [-]
WCR = 0.44
pc_b = np.where(WCR<=0.27, 13.76e6, 
                np.where(WCR>=0.84, 1.98e6, 
                         np.where(0.27<WCR<=0.43, (6.67e6-13.76e6)*(WCR-0.27)/(0.43-0.27)+13.76e6, (1.98e6-6.67e6)*(WCR-0.43)/(0.84-0.43)+6.67e6)))/1e6
# pc_b = 6.56e6/1e6                            # [Pa]
sat_m = 0.35
B_s = 30e6/1e6                                 # [Pa]
N_s = 1.2
Z_s = 0.5                                 # [℃]
T_sw = 100.0                                # [℃]
Sw0 = 0.5

# 气相和水相的相对渗透率krg krw [-]
perm_p = 1.514
perm_m = 1.572
perm_z = 31.8
A_g = 0.5
A_w = 0.5

# 混凝土有效扩散系数 Deff  [m^2/s]
fs = 1                                   # [-]
patm = 101325.0/1e6                            # [Pa]
D_va0 = 2.17E-5                          # [m^2/s]
T0va = 20                                # [℃]

# 固体骨架密度rhos  [kg/m^3]
rhos0 = 2300                             # [kg/m^3]
A_rhos = 0.3                             # [kg/(m^3 ℃)]

# 混凝土热膨胀系数betas  [1/℃]
beta_s = 18.06E-6

# 多孔材料有效热熔(rhocp)eff [J/(m^3 K)]
cpgw = 1880/1e6                               # [J/(kg K)]
cpga = 1007/1e6                               # [J/(kg K)]
cps0 = 900/1e6                                # [J/(kg K)]
A_cps = 0.5/1e6                               # [J/(kg K ℃)]
cpw = 4180/1e6                                # [J/(kg K)]

# 混凝土有效导热系数lambdaeff  [W/(m K)]
lambdas0 = 1.9/1e6                        # [W/(m K)]
A_lambda = -1.667E-3/1e6                   # [W/(m K ℃)]

# 比蒸发焓h  [J/kg]
ah = 2.672E5/1e6  
bh = 0.38

# 边界条件
time_s_ini = 0 
time_s_bc = time_s_ini 
time_m_bc1 = 300
time_m_bc2 = 600
time_m_bc3 = 1200
time_e_bc = 1800 
time_n_bc = 8000
delt_bc = (time_e_bc-time_s_bc)/time_n_bc
L_b_bc = 0.0 
L_t_bc = 0.3 
T_t = 20 
T_s_b = 20 
T_e_b = 1200
T_rate_b = (T_e_b-T_s_b)/300 
pc_s_b = 13.82e6/1e6  
pc_t = 13.82e6/1e6  
pc_th_b = 8.74e8/1e6  
pc_e_b = 3.25e9/1e6  
pc_rate1_b = (pc_th_b-pc_s_b)/50
pc_rate2_b = (pc_e_b-pc_th_b)/250
pg_b = 101325/1e6  
pg_t = 101325/1e6  

th_T1 = 300 
th_pc1 = 50 
th_pc2 = 300 

# PDE
L_n = 300 
delx = (L_t_bc-L_b_bc)/L_n 
L_b = L_b_bc + delx
L_t = L_t_bc - delx
time_n = 2000
delt_pde = (time_m_bc1-time_s_bc)/time_n 
time_s = time_s_bc + delt_pde 
time_e = time_e_bc  

# 初始条件
L_n_ini = 8000
delx_ini = (L_t_bc-L_b_bc)/L_n_ini 
L_b_ini = L_b_bc + delx_ini 
L_t_ini = L_t_bc - delx_ini  
T_ini = 20 
maximum_T = 1200 

T_max = 1500

In [ ]:
def Asfunc(pc_b, B_s, T_sw, T_crit, out_T):
    As1 = np.ones_like(out_T) * pc_b
    Ast = B_s+(pc_b-B_s)*(2*((out_T-T_sw)/(T_crit-T_sw))**3-3*((out_T-T_sw)/(T_crit-T_sw))**2+1)
    As2 = np.where(Ast < 0, tf.ones_like(Ast)*1e-3, Ast)
    As = np.where(out_T <= T_sw, As1, As2)
    return As

def Esfunc(T_crit, N_s, Z_s, T20, out_T):
    T1 = np.where(out_T <= (T_crit - Z_s), out_T, np.ones_like(out_T)*1e-3)
    Es1 = ((T_crit-T20)/(T_crit-T1))**N_s
    Es0 = ((T_crit-T20)/Z_s)**N_s
    Es2 = Es0*(N_s/Z_s*out_T+1-N_s/Z_s*(T_crit-Z_s))
    Es = np.where(out_T <= T_crit - Z_s, Es1, Es2)
    return Es

def rhow_critfunc(b0, b1, b2, b3, b4, b5, a0, a1, a2, a3, a4, a5, pw1, pwrif, T_crit, out_T):
    rhow = (b0+b1*out_T+b2*out_T**2+b3*out_T**3+b4*out_T**4+b5*out_T**5)+(pw1-pwrif)*(a0+a1*out_T+a2*out_T**2+a3*out_T**3+a4*out_T**4+a5*out_T**5)
    return rhow

def drhowdT_critfunc(b1, b2, b3, b4, b5, a1, a2, a3, a4, a5, pw1, pwrif, T_crit, out_T):
    drhowdT = (b1+2*b2*out_T+3*b3*out_T**2+4*b4*out_T**3+5*b5*out_T**4+(pw1-pwrif)*(a1+2*a2*out_T+3*a3*out_T**2+4*a4*out_T**3+5*a5*out_T**4))
    return drhowdT

def rhowfunc(b0, b1, b2, b3, b4, b5, a0, a1, a2, a3, a4, a5, pw1, pwrif, T_crit, out_T):
    rhow1 = rhow_critfunc(b0, b1, b2, b3, b4, b5, a0, a1, a2, a3, a4, a5, pw1, pwrif, T_crit, out_T)
    T1 = np.ones_like(out_T) * T_crit  # Tcrit矩阵
    rhow_crit = rhow_critfunc(b0, b1, b2, b3, b4, b5, a0, a1, a2, a3, a4, a5, pw1, pwrif, T_crit, T1)
    drhowdT_crit = drhowdT_critfunc(b1, b2, b3, b4, b5, a1, a2, a3, a4, a5, pw1, pwrif, T_crit, T1)
    A_rhow = 300
    B_rhow = drhowdT_crit/(A_rhow-rhow_crit)
    C_rhow = 1/B_rhow*np.log(rhow_crit-A_rhow)
    rhow2 = A_rhow+np.exp(-B_rhow*(out_T-C_rhow-T_crit))
    rhow = np.where(out_T < T_crit, rhow1, rhow2) 
    return rhow

Tini = 20
As0 = Asfunc(pc_b, B_s, T_sw, T_crit, Tini)
Es0 = Esfunc(T_crit, N_s, Z_s, T20, Tini)
rhow0 = rhowfunc(b0, b1, b2, b3, b4, b5, a0, a1, a2, a3, a4, a5, pw1, pwrif, T_crit, Tini)

pc0 = As0/Es0*(Sw0**(-1/sat_m)-1)**(1-sat_m)
RH0 = np.exp(-pc0*molar_w/(rhow0*rgas*(Tini+Kelvin)))

In [ ]:
scale_min1 = 0.0 
scale_max1 = 1.0
scale_min2 = 0.0 
scale_max2 = 1.0 
min_max1=MinMaxScaler(feature_range=(scale_min1, scale_max1))
min_max2=MinMaxScaler(feature_range=(scale_min2, scale_max2))

In [ ]:
he = tf.keras.initializers.he_uniform()
random_uniform = tf.keras.initializers.RandomUniform()

In [ ]:
def inputs(T_crit, b0, b1, b2, b3, b4, b5,
           pw1, pwrif, a0, a1, a2, a3, a4, a5,
           c1, c2, c3, c4, c5, c6, molar_w, rgas, Kelvin,
           molar_a, T0, etagw0, etaga0, agw, aga, bga, bg,
           aw, bw, T_etaw, poro0, A_n, akint, bkint, bsf,
           pc_b, sat_m, B_s, N_s, Z_s, T_sw, perm_p, perm_m, perm_z, A_g, A_w,
           fs, patm, D_va0, T0va, rhos0, A_rhos, beta_s,
           cpgw, cpga, cps0, A_cps, cpw,
           lambdas0, A_lambda, ah, bh, Db0, T_Db0,
           A_pc, A_pg, A_exp,
           L_b, L_t, L_n, L_b_bc, L_t_bc, L_b_ini, L_t_ini, L_n_ini,
           time_s, time_e, time_n,
           T_s_b, T_e_b, T_t, T_rate_b,
           pc_s_b, pc_e_b, pc_t, pc_rate1_b, pc_rate2_b, th_pc1, th_pc2, th_T1,
           pg_b, pg_t,
           T_ini, pc_ini, pg_ini, time_s_ini,
           time_s_bc, time_e_bc, time_n_bc):

    x_bc = np.linspace(L_b_bc, L_t_bc, 2) 
    t_bc = np.linspace(time_s_bc, time_e_bc, time_n_bc)  
    bc_num = x_bc.shape[0] * t_bc.shape[0]  
    input_data_bc_np = np.array(np.meshgrid(x_bc, t_bc), dtype=np.float64).T.reshape(-1, 2)
    input_data_bc = tf.Variable(input_data_bc_np)
    input_data_bc_np_normalized = min_max1.fit_transform(input_data_bc_np)
    input_data_bc_np_normalized2 = min_max2.fit_transform(input_data_bc_np)
    input_data_bc_np_normalized[:, 1] = input_data_bc_np_normalized2[:, 1]  
    input_data_bc_normalized = tf.Variable(input_data_bc_np_normalized)

    T_e_bv = np.where(t_bc<=th_T1, t_bc * T_rate_b + T_s_b, T_e_b+t_bc*0)  
    T_e_tv = t_bc * 0 + T_t
    yT_bc = np.append(T_e_bv, T_e_tv, axis=0)  
    yT_bc = np.reshape(yT_bc, (yT_bc.shape[0], 1))
    
    pc_e_bv = np.where(t_bc <= th_pc1, t_bc * pc_rate1_b + pc_s_b, np.where(t_bc<=th_pc2, (t_bc - th_pc1) * pc_rate2_b + pc_th_b, pc_e_b+t_bc*0))
    pc_e_tv = t_bc * 0 + pc_t
    ypc_bc = np.append(pc_e_bv, pc_e_tv, axis=0)
    ypc_bc = np.reshape(ypc_bc, (ypc_bc.shape[0], 1))

    pg_e_bv = t_bc * 0 + pg_b
    pg_e_tv = t_bc * 0 + pg_t
    ypg_bc = np.append(pg_e_bv, pg_e_tv, axis=0)
    ypg_bc = np.reshape(ypg_bc, (ypg_bc.shape[0], 1))

    # internal points
    x_pde = np.linspace(L_b, L_t, L_n)
    t_pde = np.linspace(time_s, time_e, time_n)  

    coll_num = x_pde.shape[0] * t_pde.shape[0] 
    input_data_pde_np = np.array(np.meshgrid(x_pde, t_pde), dtype=np.float64).T.reshape(-1, 2)
    input_data_pde = tf.Variable(input_data_pde_np)
    input_data_pde_np_normalized = min_max1.transform(input_data_pde_np)
    input_data_pde_np_normalized1 = min_max2.transform(input_data_pde_np)
    input_data_pde_np_normalized[:, 1] = input_data_pde_np_normalized1[:, 1]
    input_data_pde_normalized = tf.Variable(input_data_pde_np_normalized)

    # initial  points 
    x_ini = np.linspace(L_b_ini, L_t_ini, L_n_ini)  
    t_ini = np.linspace(time_s_ini, time_s_ini, 1)  
    ini_num = x_ini.shape[0] * t_ini.shape[0]  
    input_data_ini_np = np.array(np.meshgrid(x_ini, t_ini), dtype=np.float64).T.reshape(-1, 2)
    input_data_ini = tf.Variable(input_data_ini_np)
    input_data_ini_np_normalized= min_max1.transform(input_data_ini_np)
    input_data_ini_np_normalized1 = min_max2.transform(input_data_ini_np)
    input_data_ini_np_normalized[:, 1] = input_data_ini_np_normalized1[:, 1]
    input_data_ini_normalized = tf.Variable(input_data_ini_np_normalized)
    yT_ini = np.ones([ini_num, 1]) * T_ini  
    ypc_ini = np.ones([ini_num, 1]) * pc_ini 
    ypg_ini = np.ones([ini_num, 1]) * pg_ini 

    input_data_bcini = tf.Variable(kb.concatenate((input_data_bc, input_data_ini), axis=0))
    input_data_bcini_normalized = tf.Variable(kb.concatenate((input_data_bc_normalized, input_data_ini_normalized), axis=0))
    yT_bcini = np.append(yT_bc, yT_ini, axis=0)
    ypc_bcini = np.append(ypc_bc, ypc_ini, axis=0)
    ypg_bcini = np.append(ypg_bc, ypg_ini, axis=0)

    input_data_all_np = kb.concatenate((input_data_pde_np, input_data_ini_np, input_data_bc_np), axis=0)
    input_data_all_np_normalized = kb.concatenate((input_data_pde_np_normalized, input_data_bc_np_normalized, input_data_ini_np_normalized), axis=0)
    input_data_all = kb.concatenate((input_data_pde, input_data_bc, input_data_ini), axis=0)
    input_data_all_normalized = kb.concatenate((input_data_pde_normalized, input_data_bc_normalized, input_data_ini_normalized), axis=0)

    scaler1 = np.array([1, 1]).reshape(1, 2)
    scaler1 = min_max1.transform(scaler1)  
    scaler2 = np.array([1, 1]).reshape(1, 2)
    scaler2 = min_max2.transform(scaler2)

    input_x = input_data_all_np[:, 0]
    input_t = input_data_all_np[:, 1]
    input_x_normalized = input_data_all_np_normalized[:, 0]
    input_t_normalized = input_data_all_np_normalized[:, 1]

    data_num = bc_num + ini_num + coll_num
    target_data_all_np = np.zeros((data_num, 1))

    all_input_data = {'input_x': input_x,
                      'input_t': input_t,
                      'input_x_normalized': input_x_normalized,
                      'input_t_normalized': input_t_normalized,
                      'target_data_all_np': target_data_all_np,
                      'T_ini': T_ini,
                      'pc_ini': pc_ini,
                      'pg_ini': pg_ini,
                      'T_crit': T_crit,
                      'Kelvin': Kelvin,
                      'b0': b0,
                      'b1': b1,
                      'b2': b2,
                      'b3': b3,
                      'b4': b4,
                      'b5': b5,
                      'pw1': pw1,
                      'pwrif': pwrif,
                      'a0': a0,
                      'a1': a1,
                      'a2': a2,
                      'a3': a3,
                      'a4': a4,
                      'a5': a5,
                      'c1': c1,
                      'c2': c2,
                      'c3': c3,
                      'c4': c4,
                      'c5': c5,
                      'c6': c6,
                      'molar_w': molar_w,
                      'rgas': rgas,
                      'molar_a': molar_a,
                      'T0': T0,
                      'etagw0': etagw0,
                      'etaga0': etaga0,
                      'agw': agw,
                      'aga': aga,
                      'bga': bga,
                      'bg': bg,
                      'aw': aw,
                      'bw': bw,
                      'T_etaw': T_etaw,
                      'poro0': poro0,
                      'A_n': A_n,
                      'akint': akint,
                      'bkint': bkint,
                      'bsf': bsf,
                      'pc_b': pc_b,
                      'sat_m': sat_m,
                      'B_s': B_s,
                      'N_s': N_s,
                      'Z_s': Z_s,
                      'T_sw': T_sw,
                      'A_g': A_g,
                      'A_w': A_w,
                      'fs': fs,
                      'patm': patm,
                      'D_va0': D_va0,
                      'T0va': T0va,
                      'rhos0': rhos0,
                      'A_rhos': A_rhos,
                      'beta_s': beta_s,
                      'cpgw': cpgw,
                      'cpga': cpga,
                      'cps0': cps0,
                      'A_cps': A_cps,
                      'cpw': cpw,
                      'lambdas0': lambdas0,
                      'A_lambda': A_lambda,
                      'ah': ah,
                      'bh': bh,
                      'Db0': Db0,
                      'T_Db0': T_Db0,
                      'A_pc': A_pc,
                      'A_exp': A_exp,
                      'A_pg': A_pg,
                      'input_data_pde': input_data_pde,
                      'input_data_bc': input_data_bc,
                      'input_data_ini': input_data_ini,
                      'input_data_bcini': input_data_bcini,
                      'input_data_all': input_data_all,
                      'input_data_pde': input_data_pde,
                      'input_data_bc': input_data_bc,
                      'input_data_ini': input_data_ini,
                      'input_data_bcini': input_data_bcini,
                      'input_data_all': input_data_all,
                      'yT_bc': yT_bc, 'yT_ini': yT_ini, 'yT_bcini': yT_bcini,
                      'ypc_bc': ypc_bc, 'ypc_ini': ypc_ini, 'ypc_bcini': ypc_bcini,
                      'ypg_bc': ypg_bc, 'ypg_ini': ypg_ini, 'ypg_bcini': ypg_bcini,
                      'yT_bc': yT_bc,
                      'yT_ini': yT_ini,
                      'yT_bcini': yT_bcini,
                      'ypc_bc': ypc_bc,
                      'ypc_ini': ypc_ini,
                      'ypc_bcini': ypc_bcini,
                      'ypg_bc': ypg_bc,
                      'ypg_ini': ypg_ini,
                      'ypg_bcini': ypg_bcini,
                      'bc_num': bc_num, 'x_bc': x_bc, 't_bc': t_bc,
                      'ini_num': ini_num, 'x_ini': x_ini, 't_ini': t_ini,
                      'coll_num': coll_num, 'x_pde': x_pde, 't_pde': t_pde,
                      'scaler1': scaler1, 'scaler2': scaler2, 'data_num': data_num}

    return all_input_data

In [ ]:
all_input_data = \
inputs(T_crit, b0, b1, b2, b3, b4, b5,
          pw1, pwrif, a0, a1, a2,a3, a4, a5,
          c1, c2, c3, c4, c5, c6, molar_w, rgas, Kelvin,
          molar_a, T0, etagw0, etaga0, agw, aga, bga, bg,
          aw, bw, T_etaw, poro0, A_n, akint, bkint, bsf,
          pc_b, sat_m, B_s, N_s, Z_s, T_sw, perm_p, perm_m, perm_z, A_g, A_w,
          fs, patm, D_va0, T0va, rhos0, A_rhos, beta_s,
          cpgw, cpga, cps0, A_cps, cpw,
          lambdas0, A_lambda, ah, bh, Db0, T_Db0,
          A_pc, A_pg, A_exp,
          L_b, L_t, L_n, L_b_bc, L_t_bc, L_b_ini, L_t_ini, L_n_ini,
          time_s, time_e, time_n,
          T_s_b, T_e_b, T_t, T_rate_b,
          pc_s_b, pc_e_b, pc_t, pc_rate1_b, pc_rate2_b, th_pc1, th_pc2, th_T1,
          pg_b, pg_t,
          T_ini, pc_ini, pg_ini, time_s_ini,
          time_s_bc, time_e_bc, time_n_bc)

T_ini = all_input_data['T_ini']
pc_ini = all_input_data['pc_ini']
pg_ini = all_input_data['pg_ini']
T_crit = all_input_data['T_crit']
Kelvin = all_input_data['Kelvin']
b0 = all_input_data['b0']
b1 = all_input_data['b1']
b2 = all_input_data['b2']
b3 = all_input_data['b3']
b4 = all_input_data['b4']
b5 = all_input_data['b5']
pw1 = all_input_data['pw1']
pwrif = all_input_data['pwrif']
a0 = all_input_data['a0']
a1 = all_input_data['a1']
a2 = all_input_data['a2']
a3 = all_input_data['a3']
a4 = all_input_data['a4']
a5 = all_input_data['a5']
c1 = all_input_data['c1']
c2 = all_input_data['c2']
c3 = all_input_data['c3']
c4 = all_input_data['c4']
c5 = all_input_data['c5']
c6 = all_input_data['c6']
molar_w = all_input_data['molar_w']
rgas = all_input_data['rgas']
molar_a = all_input_data['molar_a']
T0 = all_input_data['T0']
etagw0 = all_input_data['etagw0']
etaga0 = all_input_data['etaga0']
agw = all_input_data['agw']
aga = all_input_data['aga']
bga = all_input_data['bga']
bg = all_input_data['bg']
aw = all_input_data['aw']
bw = all_input_data['bw']
T_etaw = all_input_data['T_etaw']
poro0 = all_input_data['poro0']
A_n = all_input_data['A_n']
akint = all_input_data['akint']
bkint = all_input_data['bkint']
bsf = all_input_data['bsf']
pc_b = all_input_data['pc_b']
sat_m = all_input_data['sat_m']
B_s = all_input_data['B_s']
N_s = all_input_data['N_s']
Z_s = all_input_data['Z_s']
T_sw = all_input_data['T_sw']
A_g = all_input_data['A_g']
A_w = all_input_data['A_w']
fs = all_input_data['fs']
patm = all_input_data['patm']
D_va0 = all_input_data['D_va0']
T0va = all_input_data['T0va']
rhos0 = all_input_data['rhos0']
A_rhos = all_input_data['A_rhos']
beta_s = all_input_data['beta_s']
cpgw = all_input_data['cpgw']
cpga = all_input_data['cpga']
cps0 = all_input_data['cps0']
A_cps = all_input_data['A_cps']
cpw = all_input_data['cpw']
lambdas0 = all_input_data['lambdas0']
A_lambda = all_input_data['A_lambda']
ah = all_input_data['ah']
bh = all_input_data['bh']
Db0 = all_input_data['Db0']
T_Db0 = all_input_data['T_Db0']
A_pc = all_input_data['A_pc']
A_exp = all_input_data['A_exp']
A_pg = all_input_data['A_pg']
input_data_pde = all_input_data['input_data_pde']
input_data_bc = all_input_data['input_data_bc']
input_data_ini = all_input_data['input_data_ini']
input_data_bcini = all_input_data['input_data_bcini']
input_data_all = all_input_data['input_data_all']
input_data_pde = all_input_data['input_data_pde']
input_data_bc = all_input_data['input_data_bc']
input_data_ini = all_input_data['input_data_ini']
input_data_bcini = all_input_data['input_data_bcini']
input_data_all = all_input_data['input_data_all']
yT_bc = all_input_data['yT_bc']
yT_ini = all_input_data['yT_ini']
yT_bcini = all_input_data['yT_bcini']
ypc_bc = all_input_data['ypc_bc']
ypc_ini = all_input_data['ypc_ini']
ypc_bcini = all_input_data['ypc_bcini']
ypg_bc = all_input_data['ypg_bc']
ypg_ini = all_input_data['ypg_ini']
ypg_bcini = all_input_data['ypg_bcini']
yT_bc = all_input_data['yT_bc']
yT_ini = all_input_data['yT_ini']
yT_bcini = all_input_data['yT_bcini']
ypc_bc = all_input_data['ypc_bc']
ypc_ini = all_input_data['ypc_ini']
ypc_bcini = all_input_data['ypc_bcini']
ypg_bc = all_input_data['ypg_bc']
ypg_ini = all_input_data['ypg_ini']
ypg_bcini = all_input_data['ypg_bcini']

coll_num = all_input_data['coll_num']
bc_num = all_input_data['bc_num']
ini_num = all_input_data['ini_num']
data_num = all_input_data['data_num']

T_ini = all_input_data['T_ini']
pc_ini = all_input_data['pc_ini']
pg_ini = all_input_data['pg_ini']
scaler1 = all_input_data['scaler1']
scaler2 = all_input_data['scaler2']

input_train_x = all_input_data['input_x']
input_train_t = all_input_data['input_t']
input_train_x_normalized = all_input_data['input_x_normalized']
input_train_t_normalized = all_input_data['input_t_normalized']
yT_ini = all_input_data['yT_ini']
ypc_ini = all_input_data['ypc_ini']
ypg_ini = all_input_data['ypg_ini']
y_train = all_input_data['target_data_all_np']

In [ ]:
def T_PINN(input_shape):
    '''
    model input is the time 't' and location 'x'
    '''
    x_inputs = Input(shape=input_shape, name='x_inputs')
    t_inputs = Input(shape=input_shape, name='t_inputs')
    RH_inputs = Input(shape=input_shape, name='RH_inputs')
    gamma_inputs = Input(shape=input_shape, name='gamma_inputs')
    gradout_RH_inputs = Input(shape=input_shape, name='gradout_RH_inputs')
    gradout_gamma_inputs = Input(shape=input_shape, name='gradout_gamma_inputs')
    iniT_inputs = Input(shape=input_shape, name='iniT_inputs')
    RH = tf.keras.backend.stop_gradient(RH_inputs)
    gamma = tf.keras.backend.stop_gradient(gamma_inputs)
    Step_inputs = Input(shape=input_shape, name='Step_inputs')
    
    nodes = 20
    h = Concatenate(axis=1)([x_inputs, t_inputs])
    h = Dense(nodes, kernel_initializer=he, activation='tanh')(h)
    h = Dense(nodes, kernel_initializer=he, activation='tanh')(h)
    h = Dense(nodes, kernel_initializer=he, activation='tanh')(h)
    h = Dense(nodes, kernel_initializer=he, activation='tanh')(h)
    h = Dense(nodes, kernel_initializer=he, activation='tanh')(h)

    output_alpha = Dense(1, kernel_initializer=he, activation='sigmoid', name='out_alpha')(h)

    pde_T_L = Lambda(pde_T_L_func, name='pde_T_L')([x_inputs, t_inputs, output_alpha, RH, gradout_RH_inputs, Step_inputs]) 
    bcT_b_L = Lambda(bcT_b_L_func, name='bcT_b_L')([x_inputs, t_inputs, output_alpha, RH, gradout_RH_inputs, Step_inputs]) 
    bcT_t_L = Lambda(bcT_t_L_func, name='bcT_t_L')([x_inputs, t_inputs, output_alpha, RH]) 
    iniT_L = Lambda(iniT_L_func, name='iniT_L')([x_inputs, t_inputs, output_alpha]) 
    
    model = Model(inputs = [x_inputs, t_inputs, RH_inputs, gradout_RH_inputs, Step_inputs],
                  outputs = [pde_T_L, bcT_b_L, bcT_t_L, iniT_L])
    
    return model


In [ ]:
def Pc_PINN(input_shape):
    '''
    model input is the time 't' and location 'x'
    '''
    x_inputs = Input(shape=input_shape, name='x_inputs')
    t_inputs = Input(shape=input_shape, name='t_inputs')
    alpha_inputs = Input(shape=input_shape, name='alpha_inputs')
    gamma_inputs = Input(shape=input_shape, name='gamma_inputs')
    alpha = tf.keras.backend.stop_gradient(alpha_inputs)
    gamma = tf.keras.backend.stop_gradient(gamma_inputs)
    dout_alphadt_inputs = Input(shape=input_shape, name='dout_alphadt_inputs')
    gradout_gamma_inputs = Input(shape=input_shape, name='gradout_gamma_inputs')
    iniT_inputs = Input(shape=input_shape, name='iniT_inputs')
    Step_inputs = Input(shape=input_shape, name='Step_inputs')
    
    nodes = 20
    h = Concatenate(axis=1)([x_inputs, t_inputs])
    h = Dense(nodes, kernel_initializer=he, activation='tanh')(h)
    h = Dense(nodes, kernel_initializer=he, activation='tanh')(h)
    h = Dense(nodes, kernel_initializer=he, activation='tanh')(h)
    h = Dense(nodes, kernel_initializer=he, activation='tanh')(h)
    h = Dense(nodes, kernel_initializer=he, activation='tanh')(h)

    output_RH = Dense(1, kernel_initializer=he, activation='sigmoid', name='out_RH')(h)

    pde_pc_L = Lambda(pde_pc_L_func, name='pde_pc_L')([x_inputs, t_inputs, output_RH, alpha, dout_alphadt_inputs, Step_inputs])  
    bcpc_b_L = Lambda(bcpc_b_L_func, name='bcpc_b_L')([x_inputs, t_inputs, output_RH, alpha, dout_alphadt_inputs, Step_inputs]) 
    bcpc_t_L = Lambda(bcpc_t_L_func, name='bcpc_t_L')([x_inputs, t_inputs, output_RH, alpha]) 
    inipc_L = Lambda(inipc_L_func, name='inipc_L')([x_inputs, t_inputs, output_RH, alpha]) 
    
    model = Model(inputs = [x_inputs, t_inputs, alpha_inputs, dout_alphadt_inputs, Step_inputs],
                  outputs = [pde_pc_L, bcpc_b_L, bcpc_t_L, inipc_L])
    
    return model


In [ ]:
def lambdafunc(lambdas0, A_lambda, T0, out_T):
    T2 = tf.ones_like(out_T) * 800.
    temp_T = tf.where(out_T < 800., out_T, T2)
    lambdaf = lambdas0 + A_lambda*(temp_T - T0)
    return lambdaf

def dlambdadTfunc(A_lambda, out_T):
    dlambdadT = tf.where(out_T < 800., tf.ones_like(out_T) *A_lambda, tf.zeros_like(out_T))
    return dlambdadT

def porofunc(poro0, A_n, T0, out_T):
    poro = poro0 + A_n*(out_T - T0)
    return poro

def dporodTfunc(A_n):
    dporodT = A_n
    return dporodT

def rhow_critfunc(b0, b1, b2, b3, b4, b5, a0, a1, a2, a3, a4, a5, pw1, pwrif, T_crit, out_T):
    rhow = (b0+b1*out_T+b2*out_T**2+b3*out_T**3+b4*out_T**4+b5*out_T**5)+(pw1-pwrif)*(a0+a1*out_T+a2*out_T**2+a3*out_T**3+a4*out_T**4+a5*out_T**5)
    return rhow

def drhowdT_critfunc(b1, b2, b3, b4, b5, a1, a2, a3, a4, a5, pw1, pwrif, T_crit, out_T):
    drhowdT = (b1+2*b2*out_T+3*b3*out_T**2+4*b4*out_T**3+5*b5*out_T**4+(pw1-pwrif)*(a1+2*a2*out_T+3*a3*out_T**2+4*a4*out_T**3+5*a5*out_T**4))
    return drhowdT

def rhowfunc(b0, b1, b2, b3, b4, b5, a0, a1, a2, a3, a4, a5, pw1, pwrif, T_crit, out_T):
    rhow1 = rhow_critfunc(b0, b1, b2, b3, b4, b5, a0, a1, a2, a3, a4, a5, pw1, pwrif, T_crit, out_T)
    T1 = tf.ones_like(out_T) * T_crit  
    rhow_crit = rhow_critfunc(b0, b1, b2, b3, b4, b5, a0, a1, a2, a3, a4, a5, pw1, pwrif, T_crit, T1)
    drhowdT_crit = drhowdT_critfunc(b1, b2, b3, b4, b5, a1, a2, a3, a4, a5, pw1, pwrif, T_crit, T1)
    A_rhow = 300
    B_rhow = drhowdT_crit/(A_rhow-rhow_crit)
    C_rhow = 1/B_rhow*tf.math.log(rhow_crit-A_rhow)
    rhow2 = A_rhow+tf.exp(-B_rhow*(out_T-C_rhow-T_crit))
    rhow = tf.where(out_T < T_crit, rhow1, rhow2) 
    return rhow

def drhowdTfunc(b1, b2, b3, b4, b5, a1, a2, a3, a4, a5, pw1, pwrif, T_crit, out_T):
    drhowdT1 = drhowdT_critfunc(b1, b2, b3, b4, b5, a1, a2, a3, a4, a5, pw1, pwrif, T_crit, out_T)
    T1 = tf.ones_like(out_T) * T_crit  
    rhow_crit = rhow_critfunc(b0, b1, b2, b3, b4, b5, a0, a1, a2, a3, a4, a5, pw1, pwrif, T_crit, T1)
    drhowdT_crit = drhowdT_critfunc(b1, b2, b3, b4, b5, a1, a2, a3, a4, a5, pw1, pwrif, T_crit, T1)
    A_rhow = 300
    B_rhow = drhowdT_crit/(A_rhow-rhow_crit)
    C_rhow = 1/B_rhow*tf.math.log(rhow_crit-A_rhow)
    drhowdT2 = -B_rhow*tf.exp(-B_rhow*(out_T-C_rhow-T_crit))
    drhowdT = tf.where(out_T < T_crit, drhowdT1, drhowdT2) 
    return drhowdT

def rhosfunc(rhos0, A_rhos, T0, out_T):
    rhos = rhos0+A_rhos*(out_T - T0)
    return rhos

def drhosdTfunc(A_rhos):
    drhosdT = A_rhos
    return drhosdT

def Asfunc(pc_b, B_s, T_sw, T_crit, out_T):
    As1 = tf.ones_like(out_T) * pc_b
    Ast = B_s+(pc_b-B_s)*(2*((out_T-T_sw)/(T_crit-T_sw))**3-3*((out_T-T_sw)/(T_crit-T_sw))**2+1)
    As2 = tf.where(Ast < 0, tf.ones_like(Ast)*1e-3, Ast)
    As = tf.where(out_T <= T_sw, As1, As2)
    return As

def dAsdTfunc(pc_b, B_s, T_sw, T_crit, out_T):
    dAsdT1 = tf.zeros_like(out_T)
    As = Asfunc(pc_b, B_s, T_sw, T_crit, out_T)
    dAsdTt = (pc_b-B_s)*(6*(out_T-T_sw)**2/(T_crit-T_sw)**3-6*(out_T-T_sw)/(T_crit-T_sw)**2)
    dAsdT2 = tf.where(As < 0, tf.zeros_like(dAsdTt), dAsdTt)
    dAsdT = tf.where(out_T <= T_sw, dAsdT1, dAsdT2)
    return dAsdT

def Esfunc(T_crit, N_s, Z_s, T0, out_T):
    T1 = tf.where(out_T <= (T_crit - Z_s), out_T, tf.ones_like(out_T)*1e-3)
    Es1 = ((T_crit-T0)/(T_crit-T1))**N_s
    Es0 = ((T_crit-T0)/Z_s)**N_s
    Es2 = Es0*(N_s/Z_s*out_T+1-N_s/Z_s*(T_crit-Z_s))
    Es = tf.where(out_T <= T_crit - Z_s, Es1, Es2)
    return Es

def dEsdTfunc(T_crit, N_s, Z_s, T0, out_T):
    T1 = tf.where(out_T <= (T_crit - Z_s), out_T, tf.ones_like(out_T)*1e-3)
    dEsdT1 = N_s*(T_crit-T0)*((T_crit-T0)/(T_crit-T1))**(N_s-1)/(T_crit-T1)**2
    Es0 = ((T_crit - T0) / Z_s) ** N_s
    dEsdT2 = N_s/Z_s*Es0
    dEsdT = tf.where(out_T <= T_crit-Z_s, dEsdT1, dEsdT2)
    return dEsdT

def Swfunc(sat_m, out_pc, out_T):
    As = Asfunc(pc_b, B_s, T_sw, T_crit, out_T)
    Es = Esfunc(T_crit, N_s, Z_s, T0, out_T)
    Sw = (1+(out_pc*Es/As)**(1/(1-sat_m)))**(-sat_m)
    return Sw

def dSwdTfunc(sat_m, out_pc, out_T):
    As = Asfunc(pc_b, B_s, T_sw, T_crit, out_T)
    Es = Esfunc(T_crit, N_s, Z_s, T0, out_T)
    dAsdT = dAsdTfunc(pc_b, B_s, T_sw, T_crit, out_T)
    dEsdT = dEsdTfunc(T_crit, N_s, Z_s, T0, out_T)
    dSwdT = (-sat_m*(out_pc*Es/As)**(1/(1-sat_m)-1)*(1+(out_pc*Es/As)**(1/(1-sat_m)))**(-1-sat_m)*(out_pc*dEsdT/As-out_pc*Es*dAsdT/As**2)/(1-sat_m))
    return dSwdT

def dSwdpcfunc(sat_m, out_pc, out_T):
    Es = Esfunc(T_crit, N_s, Z_s, T0, out_T)
    As = Asfunc(pc_b, B_s, T_sw, T_crit, out_T)
    dSwdpc = (-Es*sat_m*(Es*out_pc/As)**(1/(1-sat_m)-1)*(1+(Es*out_pc/As)**(1/(1-sat_m)))**(-1-sat_m)/(As*(1-sat_m)))
    return dSwdpc

def pgws_critfunc(c1, c2, c3, c4, c5, c6, Kelvin, out_T):
    pgws = tf.exp(c1/(out_T+Kelvin)+c2+c3*(out_T+Kelvin)+c4*(out_T+Kelvin)**2+c5*(out_T+Kelvin)**3+c6*tf.math.log(out_T+Kelvin))/ 1e6
    return pgws

def dpgwsdT_critfunc(c1, c2, c3, c4, c5, c6, Kelvin, out_T):
    T1 = tf.ones_like(out_T) * T_crit  # Tcrit矩阵
    pgws_crit = pgws_critfunc(c1, c2, c3, c4, c5, c6, Kelvin, T1)
    dpgwsdT_crit = pgws_crit*(-c1/(out_T+Kelvin)**2+c3+c4*2*(out_T+Kelvin)+c5*3*(out_T+Kelvin)**2+c6/(out_T+Kelvin))
    return dpgwsdT_crit

def pgwsfunc(c1, c2, c3, c4, c5, c6, Kelvin, out_T):
    pgws1 = pgws_critfunc(c1, c2, c3, c4, c5, c6, Kelvin, out_T)
    T1 = tf.ones_like(out_T) * T_crit  # Tcrit矩阵
    pgws_crit = pgws_critfunc(c1, c2, c3, c4, c5, c6, Kelvin, T1)
    dpgwsdT_crit = dpgwsdT_critfunc(c1, c2, c3, c4, c5, c6, Kelvin, T1)
    A_pgws = 27.0e6
    B_pgws = dpgwsdT_crit * 1e6 / (A_pgws - pgws_crit * 1e6)
    C_pgws = 1 / B_pgws * tf.math.log(A_pgws - pgws_crit * 1e6)
    pgws2 = (A_pgws - tf.exp(-B_pgws * (out_T - C_pgws - T_crit))) / 1e6
    pgws = tf.where(out_T <= T_crit, pgws1, pgws2)
    return pgws

def pgwfunc(molar_w, rgas, Kelvin, out_pc, out_T):
    pgws = pgwsfunc(c1, c2, c3, c4, c5, c6, Kelvin, out_T)
    rhow = rhowfunc(b0, b1, b2, b3, b4, b5, a0, a1, a2, a3, a4, a5, pw1, pwrif, T_crit, out_T)
    pgw = pgws*tf.exp(-out_pc*molar_w/(rhow*rgas*(out_T+Kelvin)))
    return pgw

def dpgwdpcfunc(molar_w, rgas, Kelvin, out_pc, out_T):
    rhow = rhowfunc(b0, b1, b2, b3, b4, b5, a0, a1, a2, a3, a4, a5, pw1, pwrif, T_crit, out_T)
    pgws = pgwsfunc(c1, c2, c3, c4, c5, c6, Kelvin, out_T)
    dpgwdpc = (-tf.exp(-molar_w*out_pc/(rgas*rhow*(out_T+Kelvin)))*molar_w*pgws/(rgas*rhow*(out_T+Kelvin)))
    return dpgwdpc

def dpgwdpc2func(molar_w, rgas, Kelvin, out_pc, out_T):
    rhow = rhowfunc(b0, b1, b2, b3, b4, b5, a0, a1, a2, a3, a4, a5, pw1, pwrif, T_crit, out_T)
    pgws = pgwsfunc(c1, c2, c3, c4, c5, c6, Kelvin, out_T)
    dpgwdpc2 = (-tf.exp(-molar_w*out_pc/(rgas*rhow*(out_T+Kelvin)))*(molar_w**2)*pgws/((rgas**2)*(rhow**2)*((out_T+Kelvin)**2)))
    return dpgwdpc2

def pgafunc(out_pg, out_pc, out_T):
    pgw = pgwfunc(molar_w, rgas, Kelvin, out_pc, out_T)
    pga = out_pg-pgw
    return pga

def dpgadpcfunc(out_pc, out_T):
    dpgwdpc = dpgwdpcfunc(molar_w, rgas, Kelvin, out_pc, out_T)
    dpgadpc = -dpgwdpc
    return dpgadpc

def dpgadpgfunc():
    dpgadpg = 1
    return dpgadpg

def rhogafunc(molar_a, Kelvin, rgas, out_pg, out_pc, out_T):
    pga = pgafunc(out_pg, out_pc, out_T)
    aa = tf.ones_like(pga)*1e-3
    pga1 = tf.where(pga < 0, aa, pga)
    rhoga = molar_a*pga1/(rgas*(out_T+Kelvin))
    return rhoga

def drhogadpcfunc(molar_a, rgas, Kelvin, out_pc, out_T):
    dpgwdpc = dpgwdpcfunc(molar_w, rgas, Kelvin, out_pc, out_T)
    drhogadpc = -molar_a*dpgwdpc/(rgas*(Kelvin+out_T))
    return drhogadpc

def drhogadpgfunc(molar_a, rgas, Kelvin, out_T):
    drhogadpg = molar_a/(rgas*(Kelvin+out_T))
    return drhogadpg

def rhogwfunc(molar_w, rgas, Kelvin, out_pc, out_T):
    pgw = pgwfunc(molar_w, rgas, Kelvin, out_pc, out_T)
    rhogw = molar_w*pgw/(rgas*(out_T+Kelvin))
    return rhogw

def drhogwdpcfunc(molar_w, rgas, Kelvin, out_pc, out_T):
    dpgwdpc = dpgwdpcfunc(molar_w, rgas, Kelvin, out_pc, out_T)
    rhogwdpc = molar_w*dpgwdpc/(rgas*(Kelvin+out_T))
    return rhogwdpc

def rhogfunc(out_pg, out_pc, out_T):
    rhoga = rhogafunc(molar_a, Kelvin, rgas, out_pg, out_pc, out_T)
    rhogw = rhogwfunc(molar_w, rgas, Kelvin, out_pc, out_T)
    rhog = rhoga+rhogw
    return rhog

def drhogdpcfunc(out_pc, out_T):
    drhogwdpc = drhogwdpcfunc(molar_w, rgas, Kelvin, out_pc, out_T)
    drhogadpc = drhogadpcfunc(molar_a, rgas, Kelvin, out_pc, out_T)
    drhogdpc = drhogwdpc+drhogadpc
    return drhogdpc

def drhogdpgfunc(molar_a, rgas, Kelvin, out_T):
    dpgadpg = dpgadpgfunc()
    drhogdpg = molar_a*dpgadpg/(rgas*(Kelvin+out_T))
    return drhogdpg

def rhogcpgfunc(cpgw, cpga, out_pg, out_pc, out_T):
    rhogw = rhogwfunc(molar_w, rgas, Kelvin, out_pc, out_T)
    rhoga = rhogafunc(molar_a, Kelvin, rgas, out_pg, out_pc, out_T)
    rhogcpg = rhogw*cpgw+rhoga*cpga
    return rhogcpg

def kintfunc(akint, bkint, out_T):
    kint = akint*10**(bkint*(out_T-238.5))
    return kint

def dkintdTfunc(akint, bkint, out_T):
    kint = kintfunc(akint, bkint, out_T)
    dkintdT = kint*bkint*tf.cast(tf.math.log(10.), dtype=tf.float64)
    return dkintdT

def kfunc(bsf, out_pg, out_T):
    kint = kintfunc(akint, bkint, out_T)
    k = kint*(1+bsf/out_pg)
    return k

def dkdTfunc(akint, bkint, bsf, out_pg, out_T):
    k = kfunc(akint, bkint, out_T)
    dkdT = k*bkint*tf.cast(tf.math.log(10.), dtype=tf.float64)
    return dkdT

def dkdpgfunc(bsf, out_pg, out_T):
    kint = kintfunc(akint, bkint, out_T)
    dkdpg = -bsf*kint/(out_pg**2)
    return dkdpg

def krgfunc(A_g, perm_p, out_pc, out_T):
    Sw = Swfunc(sat_m, out_pc, out_T)
    krg = (1-Sw)**A_g*(1-Sw**(1/perm_p))**(2*perm_p)
    return krg

def dkrgdpcfunc(A_g, perm_p, out_pc, out_T):
    Sw = Swfunc(sat_m, out_pc, out_T)
    dSwdpc = dSwdpcfunc(sat_m, out_pc, out_T)
    dkrgdpc = -A_g*Sw**(A_g-1)*dSwdpc*(1-Sw**(1/perm_p))**(2*perm_p)-(1-Sw)**A_g*(2*perm_p)*(1-Sw**(1/perm_p))**(2*perm_p-1)*(1/perm_p)*Sw**(1/perm_p-1)*dSwdpc
    return dkrgdpc

def etagwfunc(etagw0, agw, T0, out_T):
    etagw = etagw0+agw*(out_T - T0)
    return etagw

def etagafunc(etaga0, aga, bga, T0, out_T):
    etaga = etaga0+aga*(out_T-T0)+bga*((out_T-T0)**2)
    return etaga

def etagfunc(bg, out_pg, out_pc, out_T):
    etagw = etagwfunc(etagw0, agw, T0, out_T)
    etaga = etagafunc(etaga0, aga, bga, T0, out_T)
    pga = pgafunc(out_pg, out_pc, out_T)
    aa = tf.ones_like(pga)*1e-3
    pga1 = tf.where(pga < 0, aa, pga)
    etag1 = etagw+(etaga-etagw)*(pga1/out_pg)**bg
    etag = tf.where(pga < 0, etagw, etag1)
    return etag

def detagdpcfunc(bg, out_pg, out_pc, out_T):
    etaga = etagafunc(etaga0, aga, bga, T0, out_T)
    etagw = etagwfunc(etagw0, agw, T0, out_T)
    pga = pgafunc(out_pg, out_pc, out_T)
    dpgadpc = dpgadpcfunc(out_pc, out_T)
    detagdpc = bg*(etaga-etagw)*(pga/out_pg)**(bg-1)*dpgadpc/out_pg
    return detagdpc

def detagdpgfunc(bg, out_pg, out_pc, out_T):
    etaga = etagafunc(etaga0, aga, bga, T0, out_T)
    etagw = etagwfunc(etagw0, agw, T0, out_T)
    pga = pgafunc(out_pg, out_pc, out_T)
    dpgadpg = dpgadpgfunc()
    detagdpg = bg*(etaga-etagw)*(pga/out_pg)**(bg-1)*(dpgadpg/out_pg-pga/(out_pg**2))
    return detagdpg

def krwfunc(A_w, perm_m, perm_z, out_pc, out_T):
    Sw = Swfunc(sat_m, out_pc, out_T)
    krw = (Sw**A_w*(1-(1-Sw**(1/perm_m))**perm_m)**2)**perm_z
    return krw

def dkrwdTfunc(A_w, perm_m, perm_z, out_pc, out_T):
    Sw = Swfunc(sat_m, out_pc, out_T)
    dSwdT = dSwdTfunc(sat_m, out_pc, out_T)
    dkrwdT = perm_z*(Sw**A_w*(1-(1-Sw**(1/perm_m))**perm_m)**2)**(perm_z-1)*(2*Sw**(A_w+1/perm_m-1)*(1-Sw**(1/perm_m))**(perm_m)*(1-(1-Sw**(1/perm_m))**perm_m)*dSwdT+A_w*Sw**(A_w-1)*(1-(1-Sw**(1/perm_m))**perm_m)**2*dSwdT)
    return dkrwdT

def dkrwdpcfunc(A_w, perm_m, perm_z, out_pc, out_T):
    Sw = Swfunc(sat_m, out_pc, out_T)
    dSwdpc = dSwdpcfunc(sat_m, out_pc, out_T)
    dkrwdpc = perm_z*(Sw**A_w*(1-(1-Sw**(1/perm_m))**perm_m)**2)**(perm_z-1)*(2*Sw**(A_w+1/perm_m-1)*(1-Sw**(1/perm_m))**(perm_m)*(1-(1-Sw**(1/perm_m))**perm_m)*dSwdpc+A_w*Sw**(A_w-1)*(1-(1-Sw**(1/perm_m))**perm_m)**2*dSwdpc)
    return dkrwdpc

def etawfunc(aw, bw, T_etaw, out_T):
    etaw = aw*(out_T+T_etaw)**bw
    return etaw

def detawdTfunc(aw, bw, T_etaw, out_T):
    detawdT = aw*bw*(out_T+T_etaw)**(bw-1)
    return detawdT

def cpsfunc(cps0, A_cps, T0, T_crit, out_T):
    T1 = tf.ones_like(out_T) * T_crit
    temp_T = tf.where(out_T < T_crit, out_T, T1)
    cps = cps0+A_cps*(temp_T-T0)
    return cps

def rhocpefffunc(cpw, out_pg, out_pc, out_T):
    poro = porofunc(poro0, A_n, T0, out_T)
    rhos = rhosfunc(rhos0, A_rhos, T0, out_T)
    cps = cpsfunc(cps0, A_cps, T0, T_crit, out_T)
    Sw = Swfunc(sat_m, out_pc, out_T)
    rhow = rhowfunc(b0, b1, b2, b3, b4, b5, a0, a1, a2, a3, a4, a5, pw1, pwrif, T_crit, out_T)
    rhogcpg = rhogcpgfunc(cpgw, cpga, out_pg, out_pc, out_T)
    rhocpeff = (1-poro)*rhos*cps+poro*(Sw*rhow*cpw+(1-Sw)*rhogcpg)
    return rhocpeff

def hfunc(ah, T_crit, bh, out_T):
    T1 = tf.where(out_T <= T_crit, out_T, tf.ones_like(out_T) * 1e-3)
    h1 = ah*(T_crit-T1)**bh
    h2 = tf.zeros_like(out_T)
    h = tf.where(out_T <= T_crit, h1, h2)
    return h

def taufunc(out_pc, out_T):
    poro = porofunc(poro0, A_n, T0, out_T)
    Sw = Swfunc(sat_m, out_pc, out_T)
    tau = poro**(1/3)*(1-Sw)**(7/3)
    return tau

def Dvafunc(D_va0, Kelvin, T0va, T_crit, patm, out_pg, out_T):
    T1 = tf.ones_like(out_T) * T_crit
    temp_T = tf.where(out_T < T_crit, out_T, T1)
    Dva = D_va0*((Kelvin+temp_T)/(Kelvin+T0va))**1.88*patm/out_pg
    return Dva

def dDvadpgfunc(patm, Kelvin, T0va, T_crit, D_va0, out_T, out_pg):
    T1 = tf.ones_like(out_T) * T_crit
    temp_T = tf.where(out_T < T_crit, out_T, T1)
    dDvadpg = -patm*((Kelvin+temp_T)/(Kelvin+T0va))**1.88*D_va0/(out_pg**2)
    return dDvadpg

def Defffunc(fs, out_pg, out_pc, out_T):
    tau = taufunc(out_pc, out_T)
    poro = porofunc(poro0, A_n, T0, out_T)
    Sw = Swfunc(sat_m, out_pc, out_T)
    Dva = Dvafunc(D_va0, Kelvin, T0va, T_crit, patm, out_pg, out_T)
    Deff = tau*poro*(1-Sw)*fs*Dva
    return Deff

def dDeffdpcfunc(fs, out_pg, out_pc, out_T):
    Dva = Dvafunc(D_va0, Kelvin, T0va, T_crit, patm, out_pg, out_T)
    poro = porofunc(poro0, A_n, T0, out_T)
    Sw = Swfunc(sat_m, out_pc, out_T)
    dSwdpc = dSwdpcfunc(sat_m, out_pc, out_T)
    dDeffdpc = -10*Dva*fs*poro**(4/3)*(1-Sw)**(7/3)*dSwdpc/3
    return dDeffdpc

def dDeffdpgfunc(fs, out_pg, out_pc, out_T):
    poro = porofunc(poro0, A_n, T0, out_T)
    Sw = Swfunc(sat_m, out_pc, out_T)
    tau = taufunc(out_pc, out_T)
    dDvadpg = dDvadpgfunc(patm, Kelvin, T0va, T_crit, D_va0, out_T, out_pg)
    dDeffdpg = fs*poro*(1-Sw)*tau*dDvadpg
    return dDeffdpg

def Mgfunc(molar_a, molar_w, out_pg, out_pc, out_T):
    pgw = pgwfunc(molar_w, rgas, Kelvin, out_pc, out_T)
    Mg1 = molar_a+(molar_w-molar_a)
    Mg2 = molar_a+(molar_w-molar_a)*pgw/out_pg
    pga = pgafunc(out_pg, out_pc, out_T)
    Mg = tf.where(pga<0, tf.ones_like(out_T)*Mg1, Mg2)
    return Mg

def dMgdpcfunc(molar_w, molar_a, out_pg, out_pc, out_T):
    dpgwdpc = dpgwdpcfunc(molar_w, rgas, Kelvin, out_pc, out_T)
    dMgdpc = (molar_w-molar_a)*dpgwdpc/out_pg
    return dMgdpc

def dMgdpgfunc(molar_w, molar_a, out_pg, out_pc, out_T):
    pgw = pgwfunc(molar_w, rgas, Kelvin, out_pc, out_T)
    dMgdpg = -(molar_w-molar_a)*pgw/(out_pg**2)    ###
    return dMgdpg

def dpgwpgdpcfunc(out_pg, out_pc, out_T):
    dpgwdpc = dpgwdpcfunc(molar_w, rgas, Kelvin, out_pc, out_T)
    dpgwpgdpc = dpgwdpc/out_pg
    return dpgwpgdpc

def dpgwpgdpc2func(out_pg, out_pc, out_T):
    dpgwdpc2 = dpgwdpc2func(molar_w, rgas, Kelvin, out_pc, out_T)
    dpgwpgdpc2 = dpgwdpc2/out_pg
    return dpgwpgdpc2

def Dbnfunc(Db0, Kelvin, T_Db0, T_crit, out_T):
    T1 = tf.ones_like(out_T) * T_crit
    temp_T = tf.where(out_T < T_crit, out_T, T1)
    Dbn = Db0*tf.exp(-(temp_T+Kelvin)/(T_Db0+Kelvin))
    return Dbn

def dDbndTfunc(Db0, Kelvin, T_Db0, out_T):
    Dbn = Dbnfunc(Db0, Kelvin, T_Db0, out_T)
    dDbndT = tf.where(out_T < T_crit, Dbn*(-1/(T_Db0+Kelvin)), tf.zeros_like(Dbn))
    return dDbndT

In [ ]:
def T_bc(t_inp, T_rate, T_s, T_e, scaler):  # 可改变温度场变化
    
    t_inp = t_inp / scaler[0,1]
    
    T_ev = tf.where(t_inp <= th_T1, t_inp * T_rate + T_s, T_e+t_inp*0)
    T_ev = tf.reshape(T_ev, [tf.shape(T_ev)[0],1])
    
    return T_ev


def pc_bc(t_inp, pc_rate1, pc_rate2, pc_s, pc_th, pc_e, scaler):
    
    t_inp = t_inp / scaler[0,1]

    pc_ev = tf.where(t_inp <= th_pc1, t_inp * pc_rate1 + pc_s, tf.where(t_inp<=th_pc2, (t_inp - th_pc1) * pc_rate2 + pc_th, pc_e+t_inp*0))
    
    pc_ev = tf.reshape(pc_ev, [tf.shape(pc_ev)[0],1])
    pc_ev = tf.cast(pc_ev, tf.float64)
    
    return pc_ev

tot_num = (bc_num+coll_num+ini_num)

In [ ]:
# T-Losses
def pde_T_L_func(LambdaList):
    
    x_inp = LambdaList[0]
    t_inp = LambdaList[1]   
    out_alpha = LambdaList[2]
    pred_RH = LambdaList[3]
    gradout_RH = LambdaList[4]
    Step = LambdaList[5]

    out_T = out_alpha*T_max
    rhow = rhowfunc(b0, b1, b2, b3, b4, b5, a0, a1, a2, a3, a4, a5, pw1, pwrif, T_crit, out_T)
    pred_pc = -rhow*rgas*(out_T + Kelvin)*tf.math.log(pred_RH+1e-8)/molar_w
    pgws = pgwsfunc(c1, c2, c3, c4, c5, c6, Kelvin, out_T)
    pgw = pgws*pred_RH
    pred_pg = pgw+101325/1e6
    
    # gradients
    gradTemp = tf.gradients(out_T, x_inp)[0]*scaler1[0,0]
    gradpc = -rhow*rgas*(out_T+Kelvin)/molar_w/pred_RH*gradout_RH*scaler1[0,0]
    gradpg = tf.gradients(pgw, x_inp)[0]*scaler1[0,0]
    dTdt = tf.gradients(out_T, t_inp)[0]*scaler2[0,1]
    
    rhocpeff = rhocpefffunc(cpw, pred_pg, pred_pc, out_T)
    rhogcpg = rhogcpgfunc(cpgw, cpga, pred_pg, pred_pc, out_T)
    k = kfunc(bsf, pred_pg, out_T)
    krg = krgfunc(A_g, perm_p, pred_pc, out_T)
    etag = etagfunc(bg, pred_pg, pred_pc, out_T)  
    kint = kintfunc(akint, bkint, out_T)
    krw = krwfunc(A_w, perm_m, perm_z, pred_pc, out_T)
    etaw = etawfunc(aw, bw, T_etaw, out_T)
    h = hfunc(ah, T_crit, bh, out_T)
    lambdaf = lambdafunc(lambdas0, A_lambda, T0, out_T)
    poro = porofunc(poro0, A_n, T0, out_T)    
    rhos = rhosfunc(rhos0, A_rhos, T0, out_T)   
    Sw = Swfunc(sat_m, pred_pc, out_T)
    dSwdt = tf.gradients(Sw, t_inp)[0]*scaler2[0,1]
    drhowdt = tf.gradients(rhow, t_inp)[0]*scaler2[0,1]
    
    div_2 = tf.gradients(rhow*kint*krw/etaw*(gradpg-gradpc), x_inp)[0]*scaler1[0,0]
    lambdaeff = lambdaf*(1+4*lambdaf*rhow*Sw/(1-poro)/rhos)
    div_6 = tf.gradients(lambdaeff*gradTemp, x_inp)[0]*scaler1[0,0]
    mvap = -poro*rhow*dSwdt - poro*Sw*drhowdt + rhow*(1-poro)*Sw*beta_s*dTdt + div_2
    alpha_PDETOT = rhocpeff*dTdt - (rhogcpg*k*krg/etag*gradpg + rhow*cpw*kint*krw/etaw*(gradpg-gradpc))*gradTemp - div_6 + mvap*h
    
    # XX=1 where loss is applicable, XX=0 where loss is not applicable
    XX = 0.5 * (1.0+tf.math.sign(t_inp-scale_min2-1e-15)) \
       * 0.5 * (1.0+tf.math.sign(x_inp-scale_min1-1e-15)) \
       * 0.5 * (1.0-tf.math.sign(x_inp-scale_max1+1e-15)) 
    
    # correcting based on the nonzero number of points in the batch 
    batch_size = tf.cast(tf.size(XX), tf.float64)
    nonzero_in_batch = tf.cast(tf.math.count_nonzero(XX), tf.float64)  
    chk = tf.cast(tf.math.count_nonzero(nonzero_in_batch), tf.float64)
    nonzero_in_batch = nonzero_in_batch * chk + coll_num * (1.0-chk)
    batch_size = batch_size * chk + tot_num * (1.0-chk)
    correction =tf.math.sqrt(batch_size/nonzero_in_batch)

    L = (alpha_PDETOT) * XX  * correction
    
    return L

# first boundary condition loss
def bcT_b_L_func(LambdaList):
    
    x_inp = LambdaList[0]
    t_inp = LambdaList[1]   
    out_alpha = LambdaList[2]
    pred_RH = LambdaList[3]
    gradout_RH = LambdaList[4]
    Step = LambdaList[5]
    
    # 计算T，pg，pc
    out_T = out_alpha*T_max
    rhow = rhowfunc(b0, b1, b2, b3, b4, b5, a0, a1, a2, a3, a4, a5, pw1, pwrif, T_crit, out_T)
    pred_pc = -rhow*rgas*(out_T + Kelvin)*tf.math.log(pred_RH+1e-8)/molar_w
    pgws = pgwsfunc(c1, c2, c3, c4, c5, c6, Kelvin, out_T)
    pgw = pgws*pred_RH
    pred_pg = pgw+101325/1e6
    
    # gradients
    gradTemp = tf.gradients(out_T, x_inp)[0]*scaler1[0,0]
    gradout_pc = -rhow*rgas*(out_T+Kelvin)/molar_w/pred_RH*gradout_RH*scaler1[0,0]
    gradout_pg = tf.gradients(pred_pg, x_inp)[0]*scaler1[0,0]
    dTdt = tf.gradients(out_T, t_inp)[0]*scaler2[0,1]
    
    T_bc_b = T_bc(t_inp, T_rate_b, T_s_b, T_e_b, scaler2)
    alpha_bc_b = T_bc_b/T_max
    
    lambdaf = lambdafunc(lambdas0, A_lambda, T0, out_T)
    poro = porofunc(poro0, A_n, T0, out_T)    
    rhos = rhosfunc(rhos0, A_rhos, T0, out_T)   
    Sw = Swfunc(sat_m, pred_pc, out_T)
    lambdaeff = lambdaf*(1+4*poro*rhow*Sw/(1-poro)/rhos)
    kint = kintfunc(akint, bkint, out_T)
    krw = krwfunc(A_w, perm_m, perm_z, pred_pc, out_T)
    etaw = etawfunc(aw, bw, T_etaw, out_T)
    h = hfunc(ah, T_crit, bh, out_T)
    
    # XX=1 where loss is applicable, XX=0 where loss is not applicable
    XX = 0.5 * (1.0-tf.math.sign(x_inp-scale_min1-1e-15)) 
        
    # correcting based on the nonzero number of points in the batch 
    batch_size = tf.cast(tf.size(XX), tf.float64)
    nonzero_in_batch = tf.cast(tf.math.count_nonzero(XX), tf.float64) 
    chk = tf.cast(tf.math.count_nonzero(nonzero_in_batch), tf.float64)
    nonzero_in_batch = nonzero_in_batch * chk + bc_num * (1.0-chk)
    batch_size = batch_size * chk + tot_num * (1.0-chk)
    correction =tf.math.sqrt(batch_size/nonzero_in_batch)
    
    L = (-(lambdaeff*gradTemp - h*rhow*kint*krw/etaw*(gradout_pg-gradout_pc)) + 25/1e6*(out_T - T_bc_b)+
         0.56*5.67E-8/1e6*((out_T+273.15)**4-(T_bc_b+273.15)**4)) * XX * correction
    
    return L 

# second boundary condition loss
def bcT_t_L_func(LambdaList):
    
    x_inp = LambdaList[0]
    t_inp = LambdaList[1]   
    out_alpha = LambdaList[2]
    pred_RH = LambdaList[3]
    
    T_bc_t = T_t + t_inp * 0
    alpha_bc_t = T_bc_t/T_max
    
    out_T = out_alpha*T_max
    rhow = rhowfunc(b0, b1, b2, b3, b4, b5, a0, a1, a2, a3, a4, a5, pw1, pwrif, T_crit, out_T)
    pgws = pgwsfunc(c1, c2, c3, c4, c5, c6, Kelvin, out_T)
    pred_pc = -rhow*rgas*(out_T+Kelvin)*tf.math.log(pred_RH)/molar_w   
    lambdaf = lambdafunc(lambdas0, A_lambda, T0, out_T)
    poro = porofunc(poro0, A_n, T0, out_T)    
    rhos = rhosfunc(rhos0, A_rhos, T0, out_T)   
    Sw = Swfunc(sat_m, pred_pc, out_T)
    lambdaeff = lambdaf*(1+4*poro*rhow*Sw/(1-poro)/rhos)
    
    # XX=1 where loss is applicable, XX=0 where loss is not applicable
    XX = 0.5 * (1.0+tf.math.sign(x_inp-scale_max1+1e-15))
        
    # correcting based on the nonzero number of points in the batch 
    batch_size = tf.cast(tf.size(XX), tf.float64)
    nonzero_in_batch = tf.cast(tf.math.count_nonzero(XX), tf.float64) 
    chk = tf.cast(tf.math.count_nonzero(nonzero_in_batch), tf.float64)
    nonzero_in_batch = nonzero_in_batch * chk + bc_num * (1.0-chk)
    batch_size = batch_size * chk + tot_num * (1.0-chk)
    correction =tf.math.sqrt(batch_size/nonzero_in_batch)
    
    L = (out_alpha - alpha_bc_t) * XX * correction

    return L 

# temperature initial condition loss
def iniT_L_func(LambdaList):
    
    x_inp = LambdaList[0]
    t_inp = LambdaList[1]
    out_alpha = LambdaList[2]
    
    out_T = out_alpha*T_max
    
    T_ini_arr = t_inp * 0 + T_ini
    alpha_ini_arr = T_ini_arr/T_max
    
    # XX=1 where loss is applicable, XX=0 where loss is not applicable
    XX = 0.5 * (1.0-tf.math.sign(t_inp-scale_min2-1e-10)) \
       * 0.5 * (1.0+tf.math.sign(x_inp-scale_min1-1e-10))  \
       * 0.5 * (1.0 - tf.math.sign(x_inp-scale_max1+1e-10)) 
    
    # correcting based on the nonzero number of points in the batch
    batch_size = tf.cast(tf.size(XX), tf.float64)
    nonzero_in_batch = tf.cast(tf.math.count_nonzero(XX), tf.float64) 
    chk = tf.cast(tf.math.count_nonzero(nonzero_in_batch), tf.float64)
    nonzero_in_batch = nonzero_in_batch * chk + ini_num * (1.0-chk)
    batch_size = batch_size * chk + tot_num * (1.0-chk)
    correction =tf.math.sqrt(batch_size/nonzero_in_batch)
    
    L = (out_alpha - alpha_ini_arr) * XX * correction
   
    return L


In [ ]:
# Pc-Losses
def pde_pc_L_func(LambdaList):
    
    x_inp = LambdaList[0]
    t_inp = LambdaList[1]   
    out_RH= LambdaList[2]
    pred_alpha = LambdaList[3]
    dpred_alphadt = LambdaList[4]
    Step = LambdaList[5]
    pred_T = pred_alpha*T_max
    rhow = rhowfunc(b0, b1, b2, b3, b4, b5, a0, a1, a2, a3, a4, a5, pw1, pwrif, T_crit, pred_T)
    out_pc = -rhow*rgas*(pred_T + Kelvin)*tf.math.log(out_RH+1e-8)/molar_w
    pgws = pgwsfunc(c1, c2, c3, c4, c5, c6, Kelvin, pred_T)
    pgw = pgws*out_RH
    pred_pg = pgw+101325/1e6
    
    # gradients
    gradpc = tf.gradients(out_pc, x_inp)[0]*scaler1[0,0]
    gradpg = tf.gradients(pgw, x_inp)[0]*scaler1[0,0]
    dTdt = dpred_alphadt*T_max*scaler2[0,1]
    
    poro = porofunc(poro0, A_n, T0, pred_T)
    rhogw = rhogwfunc(molar_w, rgas, Kelvin, out_pc, pred_T)
    Sw = Swfunc(sat_m, out_pc, pred_T)
    k = kfunc(bsf, pred_pg, pred_T)
    krg = krgfunc(A_g, perm_p, out_pc, pred_T)
    etag = etagfunc(bg, pred_pg, out_pc, pred_T)
    kint = kintfunc(akint, bkint, pred_T)
    krw = krwfunc(A_w, perm_m, perm_z, out_pc, pred_T)
    etaw = etawfunc(aw, bw, T_etaw, pred_T)
    rhog = rhogfunc(pred_pg, out_pc, pred_T)
    Mg = Mgfunc(molar_a, molar_w, pred_pg, out_pc, pred_T)
    Deff = Defffunc(fs, pred_pg, out_pc, pred_T)
    dSwdt = tf.gradients(Sw, t_inp)[0]*scaler2[0,1]
    drhogwdt = tf.gradients(rhogw, t_inp)[0]*scaler2[0,1]
    drhowdT = drhowdTfunc(b1, b2, b3, b4, b5, a1, a2, a3, a4, a5, pw1, pwrif, T_crit, pred_T)
    drhowdt = drhowdT*dTdt
    gradpgw_pg = tf.gradients((pgw/pred_pg), x_inp)[0]*scaler1[0,0]
    
    div_1 = tf.gradients(rhogw*k*krg/etag*gradpg, x_inp)[0]*scaler1[0,0]
    div_2 = tf.gradients(rhow*kint*krw/etaw*(gradpg-gradpc), x_inp)[0]*scaler1[0,0]
    div_3 = tf.gradients(rhog*molar_a*molar_w/(Mg**2)*Deff*gradpgw_pg, x_inp)[0]*scaler1[0,0]
    RH_PDETOPC = poro*(rhow-rhogw)*dSwdt + poro*(1-Sw)*drhogwdt + poro*Sw*drhowdt - (1-poro)*beta_s*(rhogw+(rhow-rhogw)*Sw)*dTdt - div_1 - div_2 - div_3
    
    # XX=1 where loss is applicable, XX=0 where loss is not applicable
    XX = 0.5 * (1.0+tf.math.sign(t_inp-scale_min2-1e-15)) \
       * 0.5 * (1.0+tf.math.sign(x_inp-scale_min1-1e-15)) \
       * 0.5 * (1.0-tf.math.sign(x_inp-scale_max1+1e-15)) 
    
    # correcting based on the nonzero number of points in the batch t
    batch_size = tf.cast(tf.size(XX), tf.float64)
    nonzero_in_batch = tf.cast(tf.math.count_nonzero(XX), tf.float64) 
    chk = tf.cast(tf.math.count_nonzero(nonzero_in_batch), tf.float64)
    nonzero_in_batch = nonzero_in_batch * chk + coll_num * (1.0-chk)
    batch_size = batch_size * chk + tot_num * (1.0-chk)
    correction =tf.math.sqrt(batch_size/nonzero_in_batch)
    
    L = (RH_PDETOPC) * XX  * correction
    
    return L

# first boundary condition loss
def bcpc_b_L_func(LambdaList):
    
    x_inp = LambdaList[0]
    t_inp = LambdaList[1]   
    out_RH = LambdaList[2]
    pred_alpha = LambdaList[3]
    dpred_alphadt = LambdaList[4]
    Step = LambdaList[5]
    
    T_bc_b = T_bc(t_inp, T_rate_b, T_s_b, T_e_b, scaler2)
    pgws_bc_b = pgwsfunc(c1, c2, c3, c4, c5, c6, Kelvin, T_bc_b)
    RH_bc_b = (1020/1e6)/pgws_bc_b
    rhogw_bc_b = molar_w*(1020/1e6)/(rgas*(T_bc_b+Kelvin))
    
    pred_T = pred_alpha*T_max
    rhow = rhowfunc(b0, b1, b2, b3, b4, b5, a0, a1, a2, a3, a4, a5, pw1, pwrif, T_crit, pred_T)
    out_pc = -rhow*rgas*(pred_T + Kelvin)*tf.math.log(out_RH+1e-8)/molar_w
    pgws = pgwsfunc(c1, c2, c3, c4, c5, c6, Kelvin, pred_T)
    pgw = pgws*out_RH
    pred_pg = pgw+101325/1e6
    
    gradpc = tf.gradients(out_pc, x_inp)[0]*scaler1[0,0]
    gradpg = tf.gradients(pgw, x_inp)[0]*scaler1[0,0]

    pred_rhogw = molar_w*pgw/(rgas*(pred_T+Kelvin))
    k = kfunc(bsf, pred_pg, pred_T)
    kint = kintfunc(akint, bkint, pred_T)
    krw = krwfunc(A_w, perm_m, perm_z, out_pc, pred_T)
    krg = krgfunc(A_g, perm_p, out_pc, pred_T)
    etag = etagfunc(bg, pred_pg, out_pc, pred_T)
    etaw = etawfunc(aw, bw, T_etaw, pred_T)
    rhog = rhogfunc(pred_pg, out_pc, pred_T)
    Mg = Mgfunc(molar_a, molar_w, pred_pg, out_pc, pred_T)
    Deff = Defffunc(fs, pred_pg, out_pc, pred_T)
    gradpgw_pg = tf.gradients((pgw/pred_pg), x_inp)[0]*scaler1[0,0]
    
    # XX=1 where loss is applicable, XX=0 where loss is not applicable
    XX = 0.5 * (1.0-tf.math.sign(x_inp-scale_min1-1e-15)) 
        
    # correcting based on the nonzero number of points in the batch 
    batch_size = tf.cast(tf.size(XX), tf.float64)
    nonzero_in_batch = tf.cast(tf.math.count_nonzero(XX), tf.float64) 
    chk = tf.cast(tf.math.count_nonzero(nonzero_in_batch), tf.float64)
    nonzero_in_batch = nonzero_in_batch * chk + bc_num * (1.0-chk)
    batch_size = batch_size * chk + tot_num * (1.0-chk)
    correction =tf.math.sqrt(batch_size/nonzero_in_batch)
    
    L = (-(pred_rhogw*k*krg/etag*gradpg+rhow*kint*krw/etaw*(gradpg-gradpc)+rhog*molar_w*molar_a/(Mg**2)*Deff* \
         gradpgw_pg)+0.025*(pred_rhogw - rhogw_bc_b))* XX * correction
    
    return L 

# second boundary condition loss
def bcpc_t_L_func(LambdaList):
    
    x_inp = LambdaList[0]
    t_inp = LambdaList[1]   
    out_RH = LambdaList[2]
    pred_alpha = LambdaList[3]
    
    RH_t = RH0
    RH_bc_t = t_inp * 0 + RH_t
   
    # XX=1 where loss is applicable, XX=0 where loss is not applicable
    XX = 0.5 * (1.0+tf.math.sign(x_inp-scale_max1+1e-15))
        
    # correcting based on the nonzero number of points in the batch 
    batch_size = tf.cast(tf.size(XX), tf.float64)
    nonzero_in_batch = tf.cast(tf.math.count_nonzero(XX), tf.float64)
    chk = tf.cast(tf.math.count_nonzero(nonzero_in_batch), tf.float64)
    nonzero_in_batch = nonzero_in_batch * chk + bc_num * (1.0-chk)
    batch_size = batch_size * chk + tot_num * (1.0-chk)
    correction =tf.math.sqrt(batch_size/nonzero_in_batch)
    
    L = (out_RH - RH_bc_t)* XX * correction

    return L 

# temperature initial condition loss
def inipc_L_func(LambdaList):
    
    x_inp = LambdaList[0]
    t_inp = LambdaList[1]
    out_RH = LambdaList[2]
    pred_alpha = LambdaList[3]
    
    RH_ini = RH0
    RH_ini_arr = t_inp * 0 + RH_ini
    
    # XX=1 where loss is applicable, XX=0 where loss is not applicable
    XX = 0.5 * (1.0-tf.math.sign(t_inp-scale_min2-1e-10)) \
       * 0.5 * (1.0+tf.math.sign(x_inp-scale_min1-1e-10)) \
       * 0.5 * (1.0-tf.math.sign(x_inp-scale_max1+1e-10))
    
    # correcting based on the nonzero number of points in the batch
    batch_size = tf.cast(tf.size(XX), tf.float64)
    nonzero_in_batch = tf.cast(tf.math.count_nonzero(XX), tf.float64)
    chk = tf.cast(tf.math.count_nonzero(nonzero_in_batch), tf.float64)
    nonzero_in_batch = nonzero_in_batch * chk + ini_num * (1.0-chk)
    batch_size = batch_size * chk + tot_num * (1.0-chk)
    correction =tf.math.sqrt(batch_size/nonzero_in_batch)
    
    L = (out_RH - RH_ini_arr)* XX * correction
   
    return L


In [ ]:
T_model = T_PINN(input_shape=(1))
T_model.summary()
tf.keras.utils.plot_model(T_model, show_shapes=True)

# finding the layer number of T output in T model
out_alpha_indx = None
for idx, layer in enumerate(T_model.layers):
    if layer.name == 'out_alpha':
        out_alpha_indx = idx
        break

In [ ]:
tf.keras.backend.clear_session()
Pc_model = Pc_PINN(input_shape=(1))
Pc_model.summary()
tf.keras.utils.plot_model(Pc_model, show_shapes=True)

# finding the layer number of pc output in Pc model
out_RH_indx = None
for idx, layer in enumerate(Pc_model.layers):
    if layer.name == 'out_RH':
        out_RH_indx = idx
        break

In [ ]:
@tf.function
def loss_grads(i, model, input_data_1, input_data_2, input_data_3, input_data_4, input_data_5):
    losses = model(inputs = [input_data_1, input_data_2, input_data_3, input_data_4, input_data_5])
    mse_losses = [tf.reduce_mean(li**2) for li in losses] # MSE loss values
    L_grads_i = tf.gradients(mse_losses[i], model.trainable_variables) # , unconnected_gradients='zero'
    trainable_count = tf.cast(model.count_params(), tf.float64) #int(np.sum([kb.count_params(p) for p in set(model.trainable_weights)]))
    
    return L_grads_i, trainable_count

def max_of_loss_grads(L_grads_i):
    max_for_network = max([tf.reduce_max(abs(L_grads_i[li])) for li in range(0, len(L_grads_i))])
    
    return max_for_network

def mean_of_loss_grads(L_grads_i, max_for_network, lambda_i_old, trainable_count):
    mean_for_each_layer = [tf.reduce_mean(abs(L_grads_i[li])) for li in range(0,len(L_grads_i))]
    mean_for_network= sum(mean_for_each_layer) / len(mean_for_each_layer) # number of paramte
    lambda_i = max_for_network / (mean_for_network * lambda_i_old)
    
    return lambda_i
    
def update_weights(lambda_i_old, lambda_i):
    lambda_i_new =  0.9 *lambda_i_old + (1-0.9) * lambda_i
    
    return lambda_i_new
            
# call back for Temp adaptive loss weights 回调Temp自适应损失权重
class MyCallback_T(tf.keras.callbacks.Callback):
    def __init__(self, pdeT_loss_weight, bcT_b_loss_weight, bcT_t_loss_weight, iniT_loss_weight):
        self.pdeT_loss_weight = pdeT_loss_weight
        self.bcT_b_loss_weight = bcT_b_loss_weight
        self.bcT_t_loss_weight = bcT_t_loss_weight
        self.iniT_loss_weight = iniT_loss_weight
        
        self.pdeT_loss_weight_history = pdeT_loss_weight_history
        self.bcT_b_loss_weight_history = bcT_b_loss_weight_history
        self.bcT_t_loss_weight_history = bcT_t_loss_weight_history
        self.iniT_loss_weight_history = iniT_loss_weight_history
        
        tf.print('++++++++++++++++++++++++++')
        tf.print('Loss weights are')
        tf.print(self.pdeT_loss_weight, self.bcT_b_loss_weight, self.bcT_t_loss_weight, self.iniT_loss_weight)
        tf.print('++++++++++++++++++++++++++')
    
    # customize your behavior
    def on_epoch_end(self, epoch, logs={}):
        
        if epoch==0:
            grads_of_pdeT, trainable_count = loss_grads(0, T_model, input_train_x_np, input_train_t_np, pred_RH, gradout_RH, Step_inputs)
            max_of_pdeT_grads = max_of_loss_grads(grads_of_pdeT)
            pdeT_ct_loss_new_weight = pdeT_loss_weight

            grads_of_bcT_b_L, trainable_count = loss_grads(1, T_model, input_train_x_np, input_train_t_np, pred_RH, gradout_RH, Step_inputs)
            bcT_b_loss_lambda = mean_of_loss_grads(grads_of_bcT_b_L, max_of_pdeT_grads, bcT_b_loss_weight, trainable_count)
            bcT_b_loss_new_weight = update_weights(bcT_b_loss_weight, bcT_b_loss_lambda)

            grads_of_bcT_t_L, trainable_count = loss_grads(2, T_model, input_train_x_np, input_train_t_np, pred_RH, gradout_RH, Step_inputs)
            bcT_t_loss_lambda = mean_of_loss_grads(grads_of_bcT_t_L, max_of_pdeT_grads, bcT_t_loss_weight, trainable_count)
            bcT_t_loss_new_weight = update_weights(bcT_t_loss_weight, bcT_t_loss_lambda)

            grads_of_iniT_L, trainable_count = loss_grads(3, T_model, input_train_x_np, input_train_t_np, pred_RH, gradout_RH, Step_inputs)
            iniT_loss_lambda = mean_of_loss_grads(grads_of_iniT_L, max_of_pdeT_grads, iniT_loss_weight, trainable_count)
            iniT_loss_new_weight = update_weights(iniT_loss_weight, iniT_loss_lambda)

            kb.set_value(self.pdeT_loss_weight, pdeT_ct_loss_new_weight)
            kb.set_value(self.bcT_b_loss_weight , bcT_b_loss_new_weight)
            kb.set_value(self.bcT_t_loss_weight , bcT_t_loss_new_weight)
            kb.set_value(self.iniT_loss_weight , iniT_loss_new_weight)

            self.pdeT_loss_weight_history.extend([pdeT_ct_loss_new_weight])
            self.bcT_b_loss_weight_history.extend([bcT_b_loss_new_weight])
            self.bcT_t_loss_weight_history.extend([bcT_t_loss_new_weight])
            self.iniT_loss_weight_history.extend([iniT_loss_new_weight])

            tf.print('++++++++++++++++++++++++++')
            tf.print('Loss weights are')
            tf.print(self.pdeT_loss_weight, self.bcT_b_loss_weight, self.bcT_t_loss_weight, self.iniT_loss_weight)
            tf.print('++++++++++++++++++++++++++')

# call back for Pc adaptive loss weights 
class MyCallback_Pc(tf.keras.callbacks.Callback):
    def __init__(self, pdepc_loss_weight, bcpc_b_loss_weight, bcpc_t_loss_weight, inipc_loss_weight):
        self.pdepc_loss_weight = pdepc_loss_weight
        self.bcpc_b_loss_weight = bcpc_b_loss_weight
        self.bcpc_t_loss_weight = bcpc_t_loss_weight
        self.inipc_loss_weight = inipc_loss_weight
        
        self.pdepc_loss_weight_history = pdepc_loss_weight_history
        self.bcpc_b_loss_weight_history = bcpc_b_loss_weight_history
        self.bcpc_t_loss_weight_history = bcpc_t_loss_weight_history
        self.inipc_loss_weight_history = inipc_loss_weight_history
    
    # customize your behavior
    def on_epoch_end(self, epoch, logs={}):
        
        if epoch==0:
            grads_of_pdepc, trainable_count = loss_grads(0, Pc_model, input_train_x_np, input_train_t_np, pred_alpha, dout_alphadt, Step_inputs)
            max_of_pdepc_grads = max_of_loss_grads(grads_of_pdepc)
            pdepc_ct_loss_new_weight = pdepc_loss_weight

            grads_of_bcpc_b_L, trainable_count = loss_grads(1, Pc_model, input_train_x_np, input_train_t_np, pred_alpha, dout_alphadt, Step_inputs)
            bcpc_b_loss_lambda = mean_of_loss_grads(grads_of_bcpc_b_L, max_of_pdepc_grads, bcpc_b_loss_weight, trainable_count)
            bcpc_b_loss_new_weight = update_weights(bcpc_b_loss_weight, bcpc_b_loss_lambda)

            grads_of_bcpc_t_L, trainable_count = loss_grads(2, Pc_model, input_train_x_np, input_train_t_np, pred_alpha, dout_alphadt, Step_inputs)
            bcpc_t_loss_lambda = mean_of_loss_grads(grads_of_bcpc_t_L, max_of_pdepc_grads, bcpc_t_loss_weight, trainable_count)
            bcpc_t_loss_new_weight = update_weights(bcpc_t_loss_weight, bcpc_t_loss_lambda)

            grads_of_inipc_L, trainable_count = loss_grads(3, Pc_model, input_train_x_np, input_train_t_np, pred_alpha, dout_alphadt, Step_inputs)
            inipc_loss_lambda = mean_of_loss_grads(grads_of_inipc_L, max_of_pdepc_grads, inipc_loss_weight, trainable_count)
            inipc_loss_new_weight = update_weights(inipc_loss_weight, inipc_loss_lambda)

            kb.set_value(self.pdepc_loss_weight, pdepc_ct_loss_new_weight)
            kb.set_value(self.bcpc_b_loss_weight , bcpc_b_loss_new_weight)
            kb.set_value(self.bcpc_t_loss_weight , bcpc_t_loss_new_weight)
            kb.set_value(self.inipc_loss_weight , inipc_loss_new_weight)

            self.pdepc_loss_weight_history.extend([pdepc_ct_loss_new_weight])
            self.bcpc_b_loss_weight_history.extend([bcpc_b_loss_new_weight])
            self.bcpc_t_loss_weight_history.extend([bcpc_t_loss_new_weight])
            self.inipc_loss_weight_history.extend([inipc_loss_new_weight])

            tf.print('++++++++++++++++++++++++++')
            tf.print('Loss weights are')
            tf.print(self.pdepc_loss_weight, self.bcpc_b_loss_weight, self.bcpc_t_loss_weight, self.inipc_loss_weight)
            tf.print('++++++++++++++++++++++++++')

#### optimizers details

In [ ]:
# for Temp
# optimizer
opt_T = tf.keras.optimizers.Adam(learning_rate=1e-3)
# initial loss weights
pdeT_loss_weight = kb.variable(1.0)
bcT_b_loss_weight = kb.variable(1.0)
bcT_t_loss_weight = kb.variable(1.0)
iniT_loss_weight = kb.variable(1.0)
# compile
T_model.compile(loss=['mse', 'mse', 'mse', 'mse'],
                loss_weights=[pdeT_loss_weight, bcT_b_loss_weight, bcT_t_loss_weight, iniT_loss_weight],
                optimizer=opt_T)

# for Pc
# optimizer
opt_pc = tf.keras.optimizers.Adam(learning_rate=1e-3)
# initial loss weights
pdepc_loss_weight = kb.variable(1.0)
bcpc_b_loss_weight = kb.variable(1.0)
bcpc_t_loss_weight = kb.variable(1.0)
inipc_loss_weight = kb.variable(1.0)
# compile
Pc_model.compile(loss=['mse', 'mse', 'mse', 'mse'],
                 loss_weights=[pdepc_loss_weight, bcpc_b_loss_weight, bcpc_t_loss_weight, inipc_loss_weight],
                 optimizer=opt_pc)

#### checkpoints

In [ ]:
T_epochs = 20 # number of epochs for T training
pc_epochs = 20 # number of epochs for Pc training

checkpoint_T = ModelCheckpoint(checkpoint_path_T,
                             save_weights_only=True,
                             monitor='pde_T_L_loss',
                             verbose=1,
                             save_best_only=False)

checkpoint_pc = ModelCheckpoint(checkpoint_path_pc,
                             save_weights_only=True,
                             monitor='pde_pc_L_loss',
                             verbose=1,
                             save_best_only=False)

#### callbacks

In [ ]:
# stop training for total loss less than a critical value 
class MyThresholdCallback(tf.keras.callbacks.Callback):
    def __init__(self, threshold):
        super(MyThresholdCallback, self).__init__()
        self.threshold = threshold

    def on_epoch_end(self, epoch, logs=None): 
        Loss = logs["loss"]
        if Loss <= self.threshold:
            self.model.stop_training = True

threshold_T = MyThresholdCallback(threshold=1e-9) 
threshold_pc = MyThresholdCallback(threshold=1e-9) 

In [ ]:
# learning rate schedule 学习速率的时间表
patience_T = 10
patience_pc = 10
patience_pg = 10

reduce_lr_T = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss',
                                                   factor=0.5, patience=patience_T,
                                                   verbose=0, mode='auto',
                                                   min_delta=0.01, cooldown=0, min_lr=1e-11)

reduce_lr_pc = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss',
                                                   factor=0.5, patience=patience_pc,
                                                   verbose=0, mode='auto',
                                                   min_delta=0.01, cooldown=0, min_lr=1e-11)

EarlyStop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=1500)

In [ ]:
# batch sizes
batch_size_T = 2048
batch_size_pc = 2048

#### sequential tranining

In [ ]:
grad_total_n = L_n + time_n_bc/time_n*2 + L_n_ini/time_n  # 520
grad_bc_n = L_n + time_n_bc/time_n*2
grad_pde_point = L_n*time_n
grad_total_n = int(grad_total_n)
time_n = int(time_n)
L_n = int(L_n)
grad_bc_n = int(grad_bc_n)
time_n_bc = int(time_n_bc)
L_n_ini = int(L_n_ini)
grad_pde_point = int(grad_pde_point)

In [ ]:
tot_iteration = 5 # total number of iterative training 
t_iteration = 4 

input_train_x_np = np.reshape(input_train_x_normalized, (input_train_x_normalized.shape[0], 1))
input_train_t_np = np.reshape(input_train_t_normalized, (input_train_t_normalized.shape[0], 1))

input_train_x_np_all = input_train_x_np.reshape(grad_total_n, time_n)
pde_input_train_x_np = input_train_x_np_all[:L_n, :]
bc_input_train_x_np = input_train_x_np_all[L_n:grad_bc_n, :].reshape(time_n_bc,2)
ini_input_train_x_np = input_train_x_np_all[grad_bc_n:grad_total_n, :].reshape(L_n_ini,1)

input_train_t_np_all = input_train_t_np.reshape(grad_total_n, time_n)
pde_input_train_t_np = input_train_t_np_all[:L_n, :]
bc_input_train_t_np = input_train_t_np_all[L_n:grad_bc_n, :].reshape(time_n_bc,2)
ini_input_train_t_np = input_train_t_np_all[grad_bc_n:grad_total_n, :].reshape(L_n_ini,1)

# start pc and pg for first iteration of Temp model training 
pred_T_b = T_bc(input_train_t_normalized, T_rate_b, T_s_b, T_e_b, scaler2)
iniT = np.ones_like(pred_T_b)*20

RH_t = RH0
gamma_b = 1020/101325
gamma_t = 0.020848
pred_rhow_b = rhowfunc(b0, b1, b2, b3, b4, b5, a0, a1, a2, a3, a4, a5, pw1, pwrif, T_crit, pred_T_b)
pred_pgws_b = pgwsfunc(c1, c2, c3, c4, c5, c6, Kelvin, pred_T_b)
pred_RH_b = 1020/1e6/pred_pgws_b
pred_RH_t = RH_t
pred_gamma_b = gamma_b * np.ones((data_num,1))
pred_gamma_t = gamma_t * np.ones((data_num,1))
x_reshaped = np.reshape(input_train_x_normalized, (input_train_x_normalized.shape[0], 1))
pred_gamma = np.ones_like(pred_T_b)*101325/1e6

pred_RH_t = RH_t* np.ones((data_num,1))
x_reshaped1 = np.where(x_reshaped>0.2, np.ones((data_num,1)), x_reshaped)
pred_RH = np.where(x_reshaped1>0.2, pred_RH_t, (1-(x_reshaped1/0.2)**(5) * 1.0) * pred_RH_b + ((x_reshaped1/0.2)**(5)* 1.0) * pred_RH_t)

pred_RH1 = np.reshape(pred_RH, (grad_total_n, time_n))
pde_pred_RH = pred_RH1[:L_n, :]
bc_pred_RH = pred_RH1[L_n:grad_bc_n, :].reshape(time_n_bc,2)
ini_pred_RH = pred_RH1[grad_bc_n:grad_total_n, :].reshape(L_n_ini,1)
        
pde_gradpred_RH = np.gradient(pde_pred_RH, axis=0).reshape(grad_pde_point,1)*scaler1[0,0]/np.gradient(pde_input_train_x_np, axis=0).reshape(grad_pde_point,1)
ini_gradpred_RH = np.gradient(ini_pred_RH, axis=0)*scaler1[0,0]/np.gradient(ini_input_train_x_np, axis=0)
bc_gradpred_RH = np.zeros((time_n_bc*2,1))
gradout_RH1 = np.vstack((pde_gradpred_RH, bc_gradpred_RH))
gradout_RH = np.zeros_like(pred_RH)

pred_gamma1 = np.reshape(pred_gamma, (grad_total_n, time_n))
pde_pred_gamma = pred_gamma1[:L_n, :]
bc_pred_gamma= pred_gamma1[L_n:grad_bc_n, :].reshape(time_n_bc,2)
ini_pred_gamma = pred_gamma1[grad_bc_n:grad_total_n, :].reshape(L_n_ini,1)
        
pde_gradpred_gamma = np.gradient(pde_pred_gamma, axis=0).reshape(grad_pde_point,1)*scaler1[0,0]/np.gradient(pde_input_train_x_np, axis=0).reshape(grad_pde_point,1)     
ini_gradpred_gamma = np.gradient(ini_pred_gamma, axis=0)*scaler1[0,0]/np.gradient(ini_input_train_x_np, axis=0)
bc_gradpred_gamma = np.zeros((time_n_bc*2,1))
gradout_gamma1 = np.vstack((pde_gradpred_gamma, bc_gradpred_gamma))
gradout_gamma = np.zeros_like(pred_gamma)

start = timeit.default_timer() 

# keeping history of losses
T_total_loss_history = []
pdeT_loss_history = []
bcT_b_loss_history = []
bcT_t_loss_history = []
iniT_loss_history = []
pc_total_loss_history = []
pdepc_loss_history = []
bcpc_b_loss_history = []
bcpc_t_loss_history = []
inipc_loss_history = []
pg_total_loss_history = []
pdepg_loss_history = []
bcpg_b_loss_history = []
bcpg_t_loss_history = []
inipg_loss_history = []

# keeping history of loss wights
pdeT_loss_weight_history = []
bcT_b_loss_weight_history = []
bcT_t_loss_weight_history = []
iniT_loss_weight_history = []
pdepc_loss_weight_history = []
bcpc_b_loss_weight_history = []
bcpc_t_loss_weight_history = []
inipc_loss_weight_history = []
pdepg_loss_weight_history = []
bcpg_b_loss_weight_history = []
bcpg_t_loss_weight_history = []
inipg_loss_weight_history = []

number_of_epochs_it_ran1 = 0
number_of_epochs_it_ran2 = 0
number_of_epochs_it_ran3 = 0
xxx = 0
yyy = 0
# zzz = 0
for i in range(0, tot_iteration):
    
    Step_inputs = (i+1)*np.ones_like(pred_T_b)
    
    if xxx == 0 or yyy == 0:
        print('\n \n \n')
        print('Step:', i+1)
        print('************************* TEMP TEMP TEMP TEMP TEMP ***********************')
        
        # in the scond iteration, recompile the alpha model (inhance the training of alpha as pred_T is more realistic now)
        if i==1:
            T_model = T_PINN(input_shape=(1))
            pdeT_loss_weight = kb.variable(1.0)
            bcT_b_loss_weight = kb.variable(1.0)
            bcT_t_loss_weight = kb.variable(1.0)
            iniT_loss_weight = kb.variable(1.0)
            T_model.compile(loss=['mse', 'mse', 'mse', 'mse'],
                               loss_weights=[pdeT_loss_weight, bcT_b_loss_weight, bcT_t_loss_weight, iniT_loss_weight],
                               optimizer=opt_T)
            
        history_T = T_model.fit({'x_inputs':input_train_x_normalized,'t_inputs':input_train_t_normalized,'RH_inputs':pred_RH, 
                                 'gradout_RH_inputs':gradout_RH, 'Step_inputs':Step_inputs},
                                {'pde_T_L':y_train, 'bcT_b_L':y_train, 'bcT_t_L':y_train,'iniT_L':y_train},
                                 batch_size=batch_size_T,
                                 epochs=T_epochs,
                                 verbose=1,
                                 callbacks=[threshold_T, reduce_lr_T, checkpoint_T, EarlyStop,
                                            MyCallback_T(pdeT_loss_weight, bcT_b_loss_weight, bcT_t_loss_weight, iniT_loss_weight)])
        
        T_total_loss_history.extend(history_T.history['loss'])
        pdeT_loss_history.extend(history_T.history['pde_T_L_loss'])
        bcT_b_loss_history.extend(history_T.history['bcT_b_L_loss'])
        bcT_t_loss_history.extend(history_T.history['bcT_t_L_loss'])
        iniT_loss_history.extend(history_T.history['iniT_L_loss'])
        
        LossT = {"pde_T_L_loss": list(history_T.history['pde_T_L_loss']),
                 "bcT_b_L_loss": list(history_T.history['bcT_b_L_loss']),
                 "bcT_t_L_loss": list(history_T.history['bcT_t_L_loss']),
                 "iniT_L_loss": list(history_T.history['iniT_L_loss'])}
        LossT = pd.DataFrame(LossT)
        pd.set_option('display.max_rows', None)
        dataT = open(history_path_T + f'{case_info_text_T}.txt', 'a+')
        print(LossT, file=dataT)
        
        inp = T_model.input  
        functors = kb.function([inp], T_model.layers[out_alpha_indx].output)
        pred_alpha = functors([input_train_x_np, input_train_t_np]) 
        test_alpha = functors([test_x, test_t]) 

        input_train_x_tf = tf.convert_to_tensor(input_train_x_np)
        input_train_t_tf = tf.convert_to_tensor(input_train_t_np)
        T_model_new = tf.keras.Model(inputs=T_model.input, outputs=T_model.layers[out_alpha_indx].output)
        with tf.GradientTape() as tape1:
            tape1.watch([input_train_x_tf, input_train_t_tf])
            outputalpha1 = T_model_new([input_train_x_tf, input_train_t_tf])
            
        with tf.GradientTape() as tape11:
            tape11.watch([input_train_x_tf, input_train_t_tf])
            outputalpha2 = T_model_new([input_train_x_tf, input_train_t_tf])
            
        dout_alphadt = tape1.gradient(outputalpha1, input_train_t_tf)
        gradout_alpha = tape11.gradient(outputalpha2, input_train_x_tf)
        
        pred_alpha = tf.convert_to_tensor(pred_alpha[:,0])  
        pred_alpha = pred_alpha.numpy()
        pred_alpha = np.reshape(pred_alpha, (pred_alpha.shape[0], 1))
        
        dout_alphadt = tf.convert_to_tensor(dout_alphadt[:,0])  
        dout_alphadt = dout_alphadt.numpy()
        dout_alphadt = np.reshape(dout_alphadt, (dout_alphadt.shape[0], 1))
        
        gradout_alpha = tf.convert_to_tensor(gradout_alpha[:,0])  
        gradout_alpha = gradout_alpha.numpy()
        gradout_alpha = np.reshape(gradout_alpha, (gradout_alpha.shape[0], 1))
        
    number_of_epochs_it_ran3 = len(history_T.history['loss'])
    xxx = number_of_epochs_it_ran3 % (3*T_epochs)

    if xxx == 0 or yyy == 0:
        print('\n \n \n')
        print('Step:', i+1)
        print('************************* PC PC PC PC PC ***********************')
        history_Pc = Pc_model.fit({'x_inputs':input_train_x_normalized,'t_inputs':input_train_t_normalized, 'alpha_inputs':pred_alpha, 
                                   'dout_alphadt_inputs':dout_alphadt, 'Step_inputs':Step_inputs},
                                    {'pde_pc_L':y_train, 'bcpc_b_L':y_train, 'bcpc_t_L':y_train,'inipc_L':y_train},
                                    batch_size=batch_size_pc,
                                    epochs=pc_epochs,
                                    verbose=1,
                                    callbacks=[threshold_pc, reduce_lr_pc, checkpoint_pc, EarlyStop,
                                               MyCallback_Pc(pdepc_loss_weight, bcpc_b_loss_weight, bcpc_t_loss_weight, inipc_loss_weight)])

        pc_total_loss_history.extend(history_Pc.history['loss'])
        pdepc_loss_history.extend(history_Pc.history['pde_pc_L_loss'])
        bcpc_b_loss_history.extend(history_Pc.history['bcpc_b_L_loss'])
        bcpc_t_loss_history.extend(history_Pc.history['bcpc_t_L_loss'])
        inipc_loss_history.extend(history_Pc.history['inipc_L_loss'])
        
        Losspc = {"pde_pc_L_loss": list(history_Pc.history['pde_pc_L_loss']),
                  "bcpc_b_L_loss": list(history_Pc.history['bcpc_b_L_loss']),
                  "bcpc_t_L_loss": list(history_Pc.history['bcpc_t_L_loss']),
                  "inipc_L_loss": list(history_Pc.history['inipc_L_loss'])}
        Losspc = pd.DataFrame(Losspc)
        pd.set_option('display.max_rows', None)
        datapc = open(history_path_pc + f'{case_info_text_pc}.txt', 'a+')
        print(Losspc, file=datapc)

        inp = Pc_model.input  
        functors = kb.function([inp], Pc_model.layers[out_RH_indx].output)
        pred_RH = functors([input_train_x_np, input_train_t_np])
        test_RH = functors([test_x, test_t]) 

        input_train_x_tf = tf.convert_to_tensor(input_train_x_np)
        input_train_t_tf = tf.convert_to_tensor(input_train_t_np)
        Pc_model_new = tf.keras.Model(inputs=Pc_model.input, outputs=Pc_model.layers[out_RH_indx].output)
        with tf.GradientTape() as tape2:
            tape2.watch([input_train_x_tf, input_train_t_tf])
            outputRH1 = Pc_model_new([input_train_x_tf, input_train_t_tf])
            
        with tf.GradientTape() as tape22:
            tape22.watch([input_train_x_tf, input_train_t_tf])
            outputRH2 = Pc_model_new([input_train_x_tf, input_train_t_tf])
            
        gradout_RH = tape2.gradient(outputRH1, input_train_x_tf)
        dout_RHdt = tape22.gradient(outputRH2, input_train_t_tf)
        
        pred_RH = tf.convert_to_tensor(pred_RH[:,0])
        pred_RH =pred_RH.numpy()
        pred_RH = np.reshape(pred_RH, (pred_RH.shape[0], 1))
        
        gradout_RH = tf.convert_to_tensor(gradout_RH[:,0])
        gradout_RH = gradout_RH.numpy()
        gradout_RH = np.reshape(gradout_RH, (gradout_RH.shape[0], 1))
        
        dout_RHdt = tf.convert_to_tensor(dout_RHdt[:,0])
        dout_RHdt = dout_RHdt.numpy()
        dout_RHdt = np.reshape(dout_RHdt, (dout_RHdt.shape[0], 1))

    number_of_epochs_it_ran2 = len(history_Pc.history['loss'])
    yyy = number_of_epochs_it_ran2 % pc_epochs


    if xxx > 0 and yyy > 0:
        break

print('SOLUTION IS DONE!')

dataT.close()
datapc.close()
# datapg.close()

stop = timeit.default_timer()

print('Run time: ', round(stop - start,2),'(s) .....', round((stop - start)/60,2), '(min)')

# Outputs

In [ ]:
# loading beast Heat model wights
best_T_model = T_PINN(input_shape=(1))
best_T_model.load_weights(checkpoint_path_T)
# loading beast Pc model wights
best_Pc_model = Pc_PINN(input_shape=(1))
best_Pc_model.load_weights(checkpoint_path_pc)

## Convergence graphs

In [ ]:
# temp loss graph
# plt.semilogy(T_total_loss_history, label='Total loss')
plt.semilogy(pdeT_loss_history, label='T_pde')
plt.semilogy(bcT_b_loss_history, label='T_bc1')
plt.semilogy(bcT_t_loss_history, label='T_bc2')
plt.semilogy(iniT_loss_history, label='T_0')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('log(loss)')
plt.legend()
plt.savefig(image_path_T + '/_'+case_info_text_T + '_loss_history.png') 

In [ ]:
# PC loss graph
# plt.semilogy(T_total_loss_history, label='Total loss')
plt.semilogy(pdepc_loss_history, label='pc_pde')
plt.semilogy(bcpc_b_loss_history, label='pc_bc1')
plt.semilogy(bcpc_t_loss_history, label='pc_bc2')
plt.semilogy(inipc_loss_history, label='pc_0')
plt.grid()
plt.xlabel('epoch')
plt.ylabel('log(loss)')
plt.legend()
plt.savefig(image_path_pc + '/_' +case_info_text_pc + '_loss_history.png') 